# **Deforum Stable Diffusion v0.5**
[Stable Diffusion](https://github.com/CompVis/stable-diffusion) by Robin Rombach, Andreas Blattmann, Dominik Lorenz, Patrick Esser, Björn Ommer and the [Stability.ai](https://stability.ai/) Team. [K Diffusion](https://github.com/crowsonkb/k-diffusion) by [Katherine Crowson](https://twitter.com/RiversHaveWings). You need to get the ckpt file and put it on your Google Drive first to use this. It can be downloaded from [HuggingFace](https://huggingface.co/CompVis/stable-diffusion).

Notebook by [deforum](https://discord.gg/upmXXsrwZc)

**Important note:** this notebook severely lacks maintainance as the most devs have moved to [the WebUI extension](https://github.com/deforum-art/deforum-for-automatic1111-webui). Please, visit the [Deforum Discord server](https://discord.gg/deforum) to get info on the more active forks

If you still want to use this notebook, **proceed only if you know what you're doing!**

By using this Notebook, you agree to the following Terms of Use, and license:

**Stablity.AI Model Terms of Use**

This model is open access and available to all, with a CreativeML OpenRAIL-M license further specifying rights and usage.

The CreativeML OpenRAIL License specifies:

You can't use the model to deliberately produce nor share illegal or harmful outputs or content
CompVis claims no rights on the outputs you generate, you are free to use them and are accountable for their use which must not go against the provisions set in the license
You may re-distribute the weights and use the model commercially and/or as a service. If you do, please be aware you have to include the same use restrictions as the ones in the license and share a copy of the CreativeML OpenRAIL-M to all your users (please read the license entirely and carefully)


Please read the full license here: https://huggingface.co/spaces/CompVis/stable-diffusion-license

# Setup

In [1]:
selected_checkpoint = "/home/jnl6052/music/model/sd-v1-4.ckpt"

'''
options:

stable diffusion v1-4: /home/jnl6052/music/model/sd-v1-4.ckpt
lexica v2: /home/jnl6052/music/model/lexica-v2.ckpt

'''

In [2]:
#@markdown **NVIDIA GPU**
import subprocess
sub_p_res = subprocess.run(['nvidia-smi', '--query-gpu=name,memory.total,memory.free', '--format=csv,noheader'], stdout=subprocess.PIPE).stdout.decode('utf-8')
print(sub_p_res)

NVIDIA GeForce RTX 2060 SUPER, 8192 MiB, 7970 MiB



In [3]:
#@markdown **Model and Output Paths**
# ask for the link
print("Local Path Variables:\n")

models_path = "/home/jnl6052/music/model" #@param {type:"string"}
output_path = "/home/jnl6052/music/output" #@param {type:"string"}

#@markdown **Google Drive Path Variables (Optional)**
mount_google_drive = False #@param {type:"boolean"}
force_remount = False

if mount_google_drive:
    from google.colab import drive # type: ignore
    try:
        drive_path = "/content/drive"
        drive.mount(drive_path,force_remount=force_remount)
        models_path_gdrive = "/content/drive/MyDrive/AI/models" #@param {type:"string"}
        output_path_gdrive = "/content/drive/MyDrive/AI/StableDiffusion" #@param {type:"string"}
        models_path = models_path_gdrive
        output_path = output_path_gdrive
    except:
        print("...error mounting drive or with drive path variables")
        print("...reverting to default path variables")

import os
# os.makedirs(models_path, exist_ok=True)
# os.makedirs(output_path, exist_ok=True)

print(f"models_path: {models_path}")
print(f"output_path: {output_path}")

Local Path Variables:

models_path: /home/jnl6052/music/model
output_path: /home/jnl6052/music/output


In [4]:
#@markdown **Setup Environment**

setup_environment = True #@param {type:"boolean"}
print_subprocess = False #@param {type:"boolean"}

if setup_environment:
    import subprocess, time
    print("Setting up environment...")
    start_time = time.time()
    all_process = [
        ['pip', 'install', 'torch==1.12.1+cu113', 'torchvision==0.13.1+cu113', '--extra-index-url', 'https://download.pytorch.org/whl/cu113'],
        ['pip', 'install', 'omegaconf==2.2.3', 'einops==0.4.1', 'pytorch-lightning==1.7.4', 'torchmetrics==0.9.3', 'torchtext==0.13.1', 'transformers==4.21.2', 'kornia==0.6.7'],
        ['git', 'clone', 'https://github.com/deforum/stable-diffusion'],
        ['pip', 'install', '-e', 'git+https://github.com/CompVis/taming-transformers.git@master#egg=taming-transformers'],
        ['pip', 'install', '-e', 'git+https://github.com/openai/CLIP.git@main#egg=clip'],
        ['pip', 'install', 'accelerate', 'ftfy', 'jsonmerge', 'matplotlib', 'resize-right', 'timm', 'torchdiffeq'],
        ['git', 'clone', 'https://github.com/shariqfarooq123/AdaBins.git'],
        ['git', 'clone', 'https://github.com/isl-org/MiDaS.git'],
        ['git', 'clone', 'https://github.com/MSFTserver/pytorch3d-lite.git'],
    ]
    for process in all_process:
        running = subprocess.run(process,stdout=subprocess.PIPE).stdout.decode('utf-8')
        if print_subprocess:
            print(running)
    
    print(subprocess.run(['git', 'clone', 'https://github.com/deforum/k-diffusion/'], stdout=subprocess.PIPE).stdout.decode('utf-8'))
    with open('k-diffusion/k_diffusion/__init__.py', 'w') as f:
        f.write('')

    end_time = time.time()
    print(f"Environment set up in {end_time-start_time:.0f} seconds")

Setting up environment...


fatal: destination path 'stable-diffusion' already exists and is not an empty directory.
  Running command git fetch -q --tags
  Running command git reset --hard -q 3ba01b241669f5ade541ce990f7650a3b8f65318
  Running command git fetch -q --tags
  Running command git reset --hard -q a9b1bf5920416aaeaec965c25dd9e8f98c864f16



Environment set up in 15 seconds


fatal: destination path 'AdaBins' already exists and is not an empty directory.
fatal: destination path 'MiDaS' already exists and is not an empty directory.
fatal: destination path 'pytorch3d-lite' already exists and is not an empty directory.
fatal: destination path 'k-diffusion' already exists and is not an empty directory.


In [5]:
#@markdown **Python Definitions**
import json
from IPython import display

import gc, math, os, pathlib, subprocess, sys, time
import cv2
import numpy as np
import pandas as pd
import random
import requests
import torch
import torch.nn as nn
import torchvision.transforms as T
import torchvision.transforms.functional as TF
from contextlib import contextmanager, nullcontext
from einops import rearrange, repeat
from omegaconf import OmegaConf
from PIL import Image
from pytorch_lightning import seed_everything
from skimage.exposure import match_histograms
from torchvision.utils import make_grid
from tqdm import tqdm, trange
from types import SimpleNamespace
from torch import autocast
import re
from scipy.ndimage import gaussian_filter

sys.path.extend([
    'src/taming-transformers',
    'src/clip',
    'stable-diffusion/',
    'k-diffusion',
    'pytorch3d-lite',
    'AdaBins',
    'MiDaS',
])

import py3d_tools as p3d

from helpers import DepthModel, sampler_fn
from k_diffusion.external import CompVisDenoiser
from ldm.util import instantiate_from_config
from ldm.models.diffusion.ddim import DDIMSampler
from ldm.models.diffusion.plms import PLMSSampler

def sanitize(prompt):
    whitelist = set('abcdefghijklmnopqrstuvwxyz ABCDEFGHIJKLMNOPQRSTUVWXYZ')
    tmp = ''.join(filter(whitelist.__contains__, prompt))
    return tmp.replace(' ', '_')

from functools import reduce
def construct_RotationMatrixHomogenous(rotation_angles):
    assert(type(rotation_angles)==list and len(rotation_angles)==3)
    RH = np.eye(4,4)
    cv2.Rodrigues(np.array(rotation_angles), RH[0:3, 0:3])
    return RH

# https://en.wikipedia.org/wiki/Rotation_matrix
def getRotationMatrixManual(rotation_angles):
	
    rotation_angles = [np.deg2rad(x) for x in rotation_angles]
    
    phi         = rotation_angles[0] # around x
    gamma       = rotation_angles[1] # around y
    theta       = rotation_angles[2] # around z
    
    # X rotation
    Rphi        = np.eye(4,4)
    sp          = np.sin(phi)
    cp          = np.cos(phi)
    Rphi[1,1]   = cp
    Rphi[2,2]   = Rphi[1,1]
    Rphi[1,2]   = -sp
    Rphi[2,1]   = sp
    
    # Y rotation
    Rgamma        = np.eye(4,4)
    sg            = np.sin(gamma)
    cg            = np.cos(gamma)
    Rgamma[0,0]   = cg
    Rgamma[2,2]   = Rgamma[0,0]
    Rgamma[0,2]   = sg
    Rgamma[2,0]   = -sg
    
    # Z rotation (in-image-plane)
    Rtheta      = np.eye(4,4)
    st          = np.sin(theta)
    ct          = np.cos(theta)
    Rtheta[0,0] = ct
    Rtheta[1,1] = Rtheta[0,0]
    Rtheta[0,1] = -st
    Rtheta[1,0] = st
    
    R           = reduce(lambda x,y : np.matmul(x,y), [Rphi, Rgamma, Rtheta]) 
    
    return R


def getPoints_for_PerspectiveTranformEstimation(ptsIn, ptsOut, W, H, sidelength):
    
    ptsIn2D      =  ptsIn[0,:]
    ptsOut2D     =  ptsOut[0,:]
    ptsOut2Dlist =  []
    ptsIn2Dlist  =  []
    
    for i in range(0,4):
        ptsOut2Dlist.append([ptsOut2D[i,0], ptsOut2D[i,1]])
        ptsIn2Dlist.append([ptsIn2D[i,0], ptsIn2D[i,1]])
    
    pin  =  np.array(ptsIn2Dlist)   +  [W/2.,H/2.]
    pout = (np.array(ptsOut2Dlist)  +  [1.,1.]) * (0.5*sidelength)
    pin  = pin.astype(np.float32)
    pout = pout.astype(np.float32)
    
    return pin, pout

def warpMatrix(W, H, theta, phi, gamma, scale, fV):
    
    # M is to be estimated
    M          = np.eye(4, 4)
    
    fVhalf     = np.deg2rad(fV/2.)
    d          = np.sqrt(W*W+H*H)
    sideLength = scale*d/np.cos(fVhalf)
    h          = d/(2.0*np.sin(fVhalf))
    n          = h-(d/2.0);
    f          = h+(d/2.0);
    
    # Translation along Z-axis by -h
    T       = np.eye(4,4)
    T[2,3]  = -h
    
    # Rotation matrices around x,y,z
    R = getRotationMatrixManual([phi, gamma, theta])
    
    
    # Projection Matrix 
    P       = np.eye(4,4)
    P[0,0]  = 1.0/np.tan(fVhalf)
    P[1,1]  = P[0,0]
    P[2,2]  = -(f+n)/(f-n)
    P[2,3]  = -(2.0*f*n)/(f-n)
    P[3,2]  = -1.0
    
    # pythonic matrix multiplication
    F       = reduce(lambda x,y : np.matmul(x,y), [P, T, R]) 
    
    # shape should be 1,4,3 for ptsIn and ptsOut since perspectiveTransform() expects data in this way. 
    # In C++, this can be achieved by Mat ptsIn(1,4,CV_64FC3);
    ptsIn = np.array([[
                 [-W/2., H/2., 0.],[ W/2., H/2., 0.],[ W/2.,-H/2., 0.],[-W/2.,-H/2., 0.]
                 ]])
    ptsOut  = np.array(np.zeros((ptsIn.shape), dtype=ptsIn.dtype))
    ptsOut  = cv2.perspectiveTransform(ptsIn, F)
    
    ptsInPt2f, ptsOutPt2f = getPoints_for_PerspectiveTranformEstimation(ptsIn, ptsOut, W, H, sideLength)
    
    # check float32 otherwise OpenCV throws an error
    assert(ptsInPt2f.dtype  == np.float32)
    assert(ptsOutPt2f.dtype == np.float32)
    M33 = cv2.getPerspectiveTransform(ptsInPt2f,ptsOutPt2f)

    return M33, sideLength

def anim_frame_warp_2d(prev_img_cv2, args, anim_args, keys, frame_idx):
    angle = keys.angle_series[frame_idx]
    zoom = keys.zoom_series[frame_idx]
    translation_x = keys.translation_x_series[frame_idx]
    translation_y = keys.translation_y_series[frame_idx]

    center = (args.W // 2, args.H // 2)
    trans_mat = np.float32([[1, 0, translation_x], [0, 1, translation_y]])
    rot_mat = cv2.getRotationMatrix2D(center, angle, zoom)
    trans_mat = np.vstack([trans_mat, [0,0,1]])
    rot_mat = np.vstack([rot_mat, [0,0,1]])
    if anim_args.flip_2d_perspective:
        perspective_flip_theta = keys.perspective_flip_theta_series[frame_idx]
        perspective_flip_phi = keys.perspective_flip_phi_series[frame_idx]
        perspective_flip_gamma = keys.perspective_flip_gamma_series[frame_idx]
        perspective_flip_fv = keys.perspective_flip_fv_series[frame_idx]
        M,sl = warpMatrix(args.W, args.H, perspective_flip_theta, perspective_flip_phi, perspective_flip_gamma, 1., perspective_flip_fv);
        post_trans_mat = np.float32([[1, 0, (args.W-sl)/2], [0, 1, (args.H-sl)/2]])
        post_trans_mat = np.vstack([post_trans_mat, [0,0,1]])
        bM = np.matmul(M, post_trans_mat)
        xform = np.matmul(bM, rot_mat, trans_mat)
    else:
        xform = np.matmul(rot_mat, trans_mat)

    return cv2.warpPerspective(
        prev_img_cv2,
        xform,
        (prev_img_cv2.shape[1], prev_img_cv2.shape[0]),
        borderMode=cv2.BORDER_WRAP if anim_args.border == 'wrap' else cv2.BORDER_REPLICATE
    )

def anim_frame_warp_3d(prev_img_cv2, depth, anim_args, keys, frame_idx):
    TRANSLATION_SCALE = 1.0/200.0 # matches Disco
    translate_xyz = [
        -keys.translation_x_series[frame_idx] * TRANSLATION_SCALE, 
        keys.translation_y_series[frame_idx] * TRANSLATION_SCALE, 
        -keys.translation_z_series[frame_idx] * TRANSLATION_SCALE
    ]
    rotate_xyz = [
        math.radians(keys.rotation_3d_x_series[frame_idx]), 
        math.radians(keys.rotation_3d_y_series[frame_idx]), 
        math.radians(keys.rotation_3d_z_series[frame_idx])
    ]
    rot_mat = p3d.euler_angles_to_matrix(torch.tensor(rotate_xyz, device=device), "XYZ").unsqueeze(0)
    result = transform_image_3d(prev_img_cv2, depth, rot_mat, translate_xyz, anim_args)
    torch.cuda.empty_cache()
    return result

def add_noise(sample: torch.Tensor, noise_amt: float) -> torch.Tensor:
    return sample + torch.randn(sample.shape, device=sample.device) * noise_amt

def get_output_folder(output_path, batch_folder):
    out_path = os.path.join(output_path,time.strftime('%Y-%m'))
    if batch_folder != "":
        out_path = os.path.join(out_path, batch_folder)
    os.makedirs(out_path, exist_ok=True)
    return out_path

def load_img(path, shape, use_alpha_as_mask=False):
    # use_alpha_as_mask: Read the alpha channel of the image as the mask image
    if path.startswith('http://') or path.startswith('https://'):
        image = Image.open(requests.get(path, stream=True).raw)
    else:
        image = Image.open(path)

    if use_alpha_as_mask:
        image = image.convert('RGBA')
    else:
        image = image.convert('RGB')

    image = image.resize(shape, resample=Image.LANCZOS)

    mask_image = None
    if use_alpha_as_mask:
      # Split alpha channel into a mask_image
      red, green, blue, alpha = Image.Image.split(image)
      mask_image = alpha.convert('L')
      image = image.convert('RGB')

    image = np.array(image).astype(np.float16) / 255.0
    image = image[None].transpose(0, 3, 1, 2)
    image = torch.from_numpy(image)
    image = 2.*image - 1.

    return image, mask_image

def load_mask_latent(mask_input, shape):
    # mask_input (str or PIL Image.Image): Path to the mask image or a PIL Image object
    # shape (list-like len(4)): shape of the image to match, usually latent_image.shape
    
    if isinstance(mask_input, str): # mask input is probably a file name
        if mask_input.startswith('http://') or mask_input.startswith('https://'):
            mask_image = Image.open(requests.get(mask_input, stream=True).raw).convert('RGBA')
        else:
            mask_image = Image.open(mask_input).convert('RGBA')
    elif isinstance(mask_input, Image.Image):
        mask_image = mask_input
    else:
        raise Exception("mask_input must be a PIL image or a file name")

    mask_w_h = (shape[-1], shape[-2])
    mask = mask_image.resize(mask_w_h, resample=Image.LANCZOS)
    mask = mask.convert("L")
    return mask

def prepare_mask(mask_input, mask_shape, mask_brightness_adjust=1.0, mask_contrast_adjust=1.0):
    # mask_input (str or PIL Image.Image): Path to the mask image or a PIL Image object
    # shape (list-like len(4)): shape of the image to match, usually latent_image.shape
    # mask_brightness_adjust (non-negative float): amount to adjust brightness of the iamge, 
    #     0 is black, 1 is no adjustment, >1 is brighter
    # mask_contrast_adjust (non-negative float): amount to adjust contrast of the image, 
    #     0 is a flat grey image, 1 is no adjustment, >1 is more contrast
    
    mask = load_mask_latent(mask_input, mask_shape)

    # Mask brightness/contrast adjustments
    if mask_brightness_adjust != 1:
        mask = TF.adjust_brightness(mask, mask_brightness_adjust)
    if mask_contrast_adjust != 1:
        mask = TF.adjust_contrast(mask, mask_contrast_adjust)

    # Mask image to array
    mask = np.array(mask).astype(np.float32) / 255.0
    mask = np.tile(mask,(4,1,1))
    mask = np.expand_dims(mask,axis=0)
    mask = torch.from_numpy(mask)

    if args.invert_mask:
        mask = ( (mask - 0.5) * -1) + 0.5
    
    mask = np.clip(mask,0,1)
    return mask

def maintain_colors(prev_img, color_match_sample, mode):
    if mode == 'Match Frame 0 RGB':
        return match_histograms(prev_img, color_match_sample, channel_axis=1) # was: multichannel=True
    elif mode == 'Match Frame 0 HSV':
        prev_img_hsv = cv2.cvtColor(prev_img, cv2.COLOR_RGB2HSV)
        color_match_hsv = cv2.cvtColor(color_match_sample, cv2.COLOR_RGB2HSV)
        matched_hsv = match_histograms(prev_img_hsv, color_match_hsv, channel_axis=1)
        return cv2.cvtColor(matched_hsv, cv2.COLOR_HSV2RGB)
    else: # Match Frame 0 LAB
        prev_img_lab = cv2.cvtColor(prev_img, cv2.COLOR_RGB2LAB)
        color_match_lab = cv2.cvtColor(color_match_sample, cv2.COLOR_RGB2LAB)
        matched_lab = match_histograms(prev_img_lab, color_match_lab, channel_axis=1)
        return cv2.cvtColor(matched_lab, cv2.COLOR_LAB2RGB)


#
# Callback functions
#
class SamplerCallback(object):
    # Creates the callback function to be passed into the samplers for each step
    def __init__(self, args, mask=None, init_latent=None, sigmas=None, sampler=None,
                  verbose=False):
        self.sampler_name = args.sampler
        self.dynamic_threshold = args.dynamic_threshold
        self.static_threshold = args.static_threshold
        self.mask = mask
        self.init_latent = init_latent 
        self.sigmas = sigmas
        self.sampler = sampler
        self.verbose = verbose

        self.batch_size = args.n_samples
        self.save_sample_per_step = args.save_sample_per_step
        self.show_sample_per_step = args.show_sample_per_step
        self.paths_to_image_steps = [os.path.join( args.outdir, f"{args.timestring}_{index:02}_{args.seed}") for index in range(args.n_samples) ]

        if self.save_sample_per_step:
            for path in self.paths_to_image_steps:
                os.makedirs(path, exist_ok=True)

        self.step_index = 0

        self.noise = None
        if init_latent is not None:
            self.noise = torch.randn_like(init_latent, device=device)

        self.mask_schedule = None
        if sigmas is not None and len(sigmas) > 0:
            self.mask_schedule, _ = torch.sort(sigmas/torch.max(sigmas))
        elif len(sigmas) == 0:
            self.mask = None # no mask needed if no steps (usually happens because strength==1.0)

        if self.sampler_name in ["plms","ddim"]: 
            if mask is not None:
                assert sampler is not None, "Callback function for stable-diffusion samplers requires sampler variable"

        if self.sampler_name in ["plms","ddim"]: 
            # Callback function formated for compvis latent diffusion samplers
            self.callback = self.img_callback_
        else: 
            # Default callback function uses k-diffusion sampler variables
            self.callback = self.k_callback_

        self.verbose_print = print if verbose else lambda *args, **kwargs: None

    def view_sample_step(self, latents, path_name_modifier=''):
        if self.save_sample_per_step or self.show_sample_per_step:
            samples = model.decode_first_stage(latents)
            if self.save_sample_per_step:
                fname = f'{path_name_modifier}_{self.step_index:05}.png'
                for i, sample in enumerate(samples):
                    sample = sample.double().cpu().add(1).div(2).clamp(0, 1)
                    sample = torch.tensor(np.array(sample))
                    grid = make_grid(sample, 4).cpu()
                    TF.to_pil_image(grid).save(os.path.join(self.paths_to_image_steps[i], fname))
            if self.show_sample_per_step:
                print(path_name_modifier)
                self.display_images(samples)
        return

    def display_images(self, images):
        images = images.double().cpu().add(1).div(2).clamp(0, 1)
        images = torch.tensor(np.array(images))
        grid = make_grid(images, 4).cpu()
        display.display(TF.to_pil_image(grid))
        return

    # The callback function is applied to the image at each step
    def dynamic_thresholding_(self, img, threshold):
        # Dynamic thresholding from Imagen paper (May 2022)
        s = np.percentile(np.abs(img.cpu()), threshold, axis=tuple(range(1,img.ndim)))
        s = np.max(np.append(s,1.0))
        torch.clamp_(img, -1*s, s)
        torch.FloatTensor.div_(img, s)

    # Callback for samplers in the k-diffusion repo, called thus:
    #   callback({'x': x, 'i': i, 'sigma': sigmas[i], 'sigma_hat': sigmas[i], 'denoised': denoised})
    def k_callback_(self, args_dict):
        self.step_index = args_dict['i']
        if self.dynamic_threshold is not None:
            self.dynamic_thresholding_(args_dict['x'], self.dynamic_threshold)
        if self.static_threshold is not None:
            torch.clamp_(args_dict['x'], -1*self.static_threshold, self.static_threshold)
        if self.mask is not None:
            init_noise = self.init_latent + self.noise * args_dict['sigma']
            is_masked = torch.logical_and(self.mask >= self.mask_schedule[args_dict['i']], self.mask != 0 )
            new_img = init_noise * torch.where(is_masked,1,0) + args_dict['x'] * torch.where(is_masked,0,1)
            args_dict['x'].copy_(new_img)

        self.view_sample_step(args_dict['denoised'], "x0_pred")

    # Callback for Compvis samplers
    # Function that is called on the image (img) and step (i) at each step
    def img_callback_(self, img, i):
        self.step_index = i
        # Thresholding functions
        if self.dynamic_threshold is not None:
            self.dynamic_thresholding_(img, self.dynamic_threshold)
        if self.static_threshold is not None:
            torch.clamp_(img, -1*self.static_threshold, self.static_threshold)
        if self.mask is not None:
            i_inv = len(self.sigmas) - i - 1
            init_noise = self.sampler.stochastic_encode(self.init_latent, torch.tensor([i_inv]*self.batch_size).to(device), noise=self.noise)
            is_masked = torch.logical_and(self.mask >= self.mask_schedule[i], self.mask != 0 )
            new_img = init_noise * torch.where(is_masked,1,0) + img * torch.where(is_masked,0,1)
            img.copy_(new_img)

        self.view_sample_step(img, "x")

def sample_from_cv2(sample: np.ndarray) -> torch.Tensor:
    sample = ((sample.astype(float) / 255.0) * 2) - 1
    sample = sample[None].transpose(0, 3, 1, 2).astype(np.float16)
    sample = torch.from_numpy(sample)
    return sample

def sample_to_cv2(sample: torch.Tensor, type=np.uint8) -> np.ndarray:
    sample_f32 = rearrange(sample.squeeze().cpu().numpy(), "c h w -> h w c").astype(np.float32)
    sample_f32 = ((sample_f32 * 0.5) + 0.5).clip(0, 1)
    sample_int8 = (sample_f32 * 255)
    return sample_int8.astype(type)

def transform_image_3d(prev_img_cv2, depth_tensor, rot_mat, translate, anim_args):
    # adapted and optimized version of transform_image_3d from Disco Diffusion https://github.com/alembics/disco-diffusion 
    w, h = prev_img_cv2.shape[1], prev_img_cv2.shape[0]

    aspect_ratio = float(w)/float(h)
    near, far, fov_deg = anim_args.near_plane, anim_args.far_plane, anim_args.fov
    persp_cam_old = p3d.FoVPerspectiveCameras(near, far, aspect_ratio, fov=fov_deg, degrees=True, device=device)
    persp_cam_new = p3d.FoVPerspectiveCameras(near, far, aspect_ratio, fov=fov_deg, degrees=True, R=rot_mat, T=torch.tensor([translate]), device=device)

    # range of [-1,1] is important to torch grid_sample's padding handling
    y,x = torch.meshgrid(torch.linspace(-1.,1.,h,dtype=torch.float32,device=device),torch.linspace(-1.,1.,w,dtype=torch.float32,device=device))
    z = torch.as_tensor(depth_tensor, dtype=torch.float32, device=device)
    xyz_old_world = torch.stack((x.flatten(), y.flatten(), z.flatten()), dim=1)

    xyz_old_cam_xy = persp_cam_old.get_full_projection_transform().transform_points(xyz_old_world)[:,0:2]
    xyz_new_cam_xy = persp_cam_new.get_full_projection_transform().transform_points(xyz_old_world)[:,0:2]

    offset_xy = xyz_new_cam_xy - xyz_old_cam_xy
    # affine_grid theta param expects a batch of 2D mats. Each is 2x3 to do rotation+translation.
    identity_2d_batch = torch.tensor([[1.,0.,0.],[0.,1.,0.]], device=device).unsqueeze(0)
    # coords_2d will have shape (N,H,W,2).. which is also what grid_sample needs.
    coords_2d = torch.nn.functional.affine_grid(identity_2d_batch, [1,1,h,w], align_corners=False)
    offset_coords_2d = coords_2d - torch.reshape(offset_xy, (h,w,2)).unsqueeze(0)

    image_tensor = rearrange(torch.from_numpy(prev_img_cv2.astype(np.float32)), 'h w c -> c h w').to(device)
    new_image = torch.nn.functional.grid_sample(
        image_tensor.add(1/512 - 0.0001).unsqueeze(0), 
        offset_coords_2d, 
        mode=anim_args.sampling_mode, 
        padding_mode=anim_args.padding_mode, 
        align_corners=False
    )

    # convert back to cv2 style numpy array
    result = rearrange(
        new_image.squeeze().clamp(0,255), 
        'c h w -> h w c'
    ).cpu().numpy().astype(prev_img_cv2.dtype)
    return result

def check_is_number(value):
    float_pattern = r'^(?=.)([+-]?([0-9]*)(\.([0-9]+))?)$'
    return re.match(float_pattern, value)

# prompt weighting with colons and number coefficients (like 'bacon:0.75 eggs:0.25')
# borrowed from https://github.com/kylewlacy/stable-diffusion/blob/0a4397094eb6e875f98f9d71193e350d859c4220/ldm/dream/conditioning.py
# and https://github.com/raefu/stable-diffusion-automatic/blob/unstablediffusion/modules/processing.py
def get_uc_and_c(prompts, model, args, frame = 0):
    prompt = prompts[0] # they are the same in a batch anyway

    # get weighted sub-prompts
    negative_subprompts, positive_subprompts = split_weighted_subprompts(
        prompt, frame, not args.normalize_prompt_weights
    )

    uc = get_learned_conditioning(model, negative_subprompts, "", args, -1)
    c = get_learned_conditioning(model, positive_subprompts, prompt, args, 1)

    return (uc, c)

def get_learned_conditioning(model, weighted_subprompts, text, args, sign = 1):
    if len(weighted_subprompts) < 1:
        log_tokenization(text, model, args.log_weighted_subprompts, sign)
        c = model.get_learned_conditioning(args.n_samples * [text])
    else:
        c = None
        for subtext, subweight in weighted_subprompts:
            log_tokenization(subtext, model, args.log_weighted_subprompts, sign * subweight)
            if c is None:
                c = model.get_learned_conditioning(args.n_samples * [subtext])
                c *= subweight
            else:
                c.add_(model.get_learned_conditioning(args.n_samples * [subtext]), alpha=subweight)
        
    return c

def parse_weight(match, frame = 0)->float:
    import numexpr
    w_raw = match.group("weight")
    if w_raw == None:
        return 1
    if check_is_number(w_raw):
        return float(w_raw)
    else:
        t = frame
        if len(w_raw) < 3:
            print('the value inside `-characters cannot represent a math function')
            return 1
        return float(numexpr.evaluate(w_raw[1:-1]))

def normalize_prompt_weights(parsed_prompts):
    if len(parsed_prompts) == 0:
        return parsed_prompts
    weight_sum = sum(map(lambda x: x[1], parsed_prompts))
    if weight_sum == 0:
        print(
            "Warning: Subprompt weights add up to zero. Discarding and using even weights instead.")
        equal_weight = 1 / max(len(parsed_prompts), 1)
        return [(x[0], equal_weight) for x in parsed_prompts]
    return [(x[0], x[1] / weight_sum) for x in parsed_prompts]

def split_weighted_subprompts(text, frame = 0, skip_normalize=False):
    """
    grabs all text up to the first occurrence of ':'
    uses the grabbed text as a sub-prompt, and takes the value following ':' as weight
    if ':' has no value defined, defaults to 1.0
    repeats until no text remaining
    """
    prompt_parser = re.compile("""
            (?P<prompt>         # capture group for 'prompt'
            (?:\\\:|[^:])+      # match one or more non ':' characters or escaped colons '\:'
            )                   # end 'prompt'
            (?:                 # non-capture group
            :+                  # match one or more ':' characters
            (?P<weight>((        # capture group for 'weight'
            -?\d+(?:\.\d+)?     # match positive or negative integer or decimal number
            )|(                 # or
            `[\S\s]*?`# a math function
            )))?                 # end weight capture group, make optional
            \s*                 # strip spaces after weight
            |                   # OR
            $                   # else, if no ':' then match end of line
            )                   # end non-capture group
            """, re.VERBOSE)
    negative_prompts = []
    positive_prompts = []
    for match in re.finditer(prompt_parser, text):
        w = parse_weight(match, frame)
        if w < 0:
            # negating the sign as we'll feed this to uc
            negative_prompts.append((match.group("prompt").replace("\\:", ":"), -w))
        elif w > 0:
            positive_prompts.append((match.group("prompt").replace("\\:", ":"), w))

    if skip_normalize:
        return (negative_prompts, positive_prompts)
    return (normalize_prompt_weights(negative_prompts), normalize_prompt_weights(positive_prompts))

# shows how the prompt is tokenized
# usually tokens have '</w>' to indicate end-of-word,
# but for readability it has been replaced with ' '
def log_tokenization(text, model, log=False, weight=1):
    if not log:
        return
    tokens    = model.cond_stage_model.tokenizer._tokenize(text)
    tokenized = ""
    discarded = ""
    usedTokens = 0
    totalTokens = len(tokens)
    for i in range(0, totalTokens):
        token = tokens[i].replace('</w>', ' ')
        # alternate color
        s = (usedTokens % 6) + 1
        if i < model.cond_stage_model.max_length:
            tokenized = tokenized + f"\x1b[0;3{s};40m{token}"
            usedTokens += 1
        else:  # over max token length
            discarded = discarded + f"\x1b[0;3{s};40m{token}"
    print(f"\n>> Tokens ({usedTokens}), Weight ({weight:.2f}):\n{tokenized}\x1b[0m")
    if discarded != "":
        print(
            f">> Tokens Discarded ({totalTokens-usedTokens}):\n{discarded}\x1b[0m"
        )

def generate(args, frame = 0, return_latent=False, return_sample=False, return_c=False):
    seed_everything(args.seed)
    os.makedirs(args.outdir, exist_ok=True)

    sampler = PLMSSampler(model) if args.sampler == 'plms' else DDIMSampler(model)
    model_wrap = CompVisDenoiser(model)
    batch_size = args.n_samples
    prompt = args.prompt
    assert prompt is not None
    data = [batch_size * [prompt]]
    precision_scope = autocast if args.precision == "autocast" else nullcontext

    init_latent = None
    mask_image = None
    init_image = None
    if args.init_latent is not None:
        init_latent = args.init_latent
    elif args.init_sample is not None:
        with precision_scope("cuda"):
            init_latent = model.get_first_stage_encoding(model.encode_first_stage(args.init_sample))
    elif args.use_init and args.init_image != None and args.init_image != '':
        init_image, mask_image = load_img(args.init_image, 
                                          shape=(args.W, args.H),  
                                          use_alpha_as_mask=args.use_alpha_as_mask)
        init_image = init_image.to(device)
        init_image = repeat(init_image, '1 ... -> b ...', b=batch_size)
        with precision_scope("cuda"):
            init_latent = model.get_first_stage_encoding(model.encode_first_stage(init_image))  # move to latent space        

    if not args.use_init and args.strength > 0 and args.strength_0_no_init:
        print("\nNo init image, but strength > 0. Strength has been auto set to 0, since use_init is False.")
        print("If you want to force strength > 0 with no init, please set strength_0_no_init to False.\n")
        args.strength = 0

    # Mask functions
    if args.use_mask:
        assert args.mask_file is not None or mask_image is not None, "use_mask==True: An mask image is required for a mask. Please enter a mask_file or use an init image with an alpha channel"
        assert args.use_init, "use_mask==True: use_init is required for a mask"
        assert init_latent is not None, "use_mask==True: An latent init image is required for a mask"


        mask = prepare_mask(args.mask_file if mask_image is None else mask_image, 
                            init_latent.shape, 
                            args.mask_contrast_adjust, 
                            args.mask_brightness_adjust)
        
        if (torch.all(mask == 0) or torch.all(mask == 1)) and args.use_alpha_as_mask:
            raise Warning("use_alpha_as_mask==True: Using the alpha channel from the init image as a mask, but the alpha channel is blank.")
        
        mask = mask.to(device)
        mask = repeat(mask, '1 ... -> b ...', b=batch_size)
    else:
        mask = None

    assert not ( (args.use_mask and args.overlay_mask) and (args.init_sample is None and init_image is None)), "Need an init image when use_mask == True and overlay_mask == True"
        
    t_enc = int((1.0-args.strength) * args.steps)

    # Noise schedule for the k-diffusion samplers (used for masking)
    k_sigmas = model_wrap.get_sigmas(args.steps)
    k_sigmas = k_sigmas[len(k_sigmas)-t_enc-1:]

    if args.sampler in ['plms','ddim']:
        sampler.make_schedule(ddim_num_steps=args.steps, ddim_eta=args.ddim_eta, ddim_discretize='fill', verbose=False)

    callback = SamplerCallback(args=args,
                            mask=mask, 
                            init_latent=init_latent,
                            sigmas=k_sigmas,
                            sampler=sampler,
                            verbose=False).callback  

    results = []
    with torch.no_grad():
        with precision_scope("cuda"):
            with model.ema_scope():
                for prompts in data:
                    if isinstance(prompts, tuple):
                        prompts = list(prompts)
                    if args.prompt_weighting:
                        uc, c = get_uc_and_c(prompts, model, args, frame)
                    else:
                        uc = model.get_learned_conditioning(batch_size * [""])
                        c = model.get_learned_conditioning(prompts)


                    if args.scale == 1.0:
                        uc = None
                    if args.init_c != None:
                        c = args.init_c

                    if args.sampler in ["klms","dpm2","dpm2_ancestral","heun","euler","euler_ancestral"]:
                        samples = sampler_fn(
                            c=c, 
                            uc=uc, 
                            args=args, 
                            model_wrap=model_wrap, 
                            init_latent=init_latent, 
                            t_enc=t_enc, 
                            device=device, 
                            cb=callback)
                    else:
                        # args.sampler == 'plms' or args.sampler == 'ddim':
                        if init_latent is not None and args.strength > 0:
                            z_enc = sampler.stochastic_encode(init_latent, torch.tensor([t_enc]*batch_size).to(device))
                        else:
                            z_enc = torch.randn([args.n_samples, args.C, args.H // args.f, args.W // args.f], device=device)
                        if args.sampler == 'ddim':
                            samples = sampler.decode(z_enc, 
                                                     c, 
                                                     t_enc, 
                                                     unconditional_guidance_scale=args.scale,
                                                     unconditional_conditioning=uc,
                                                     img_callback=callback)
                        elif args.sampler == 'plms': # no "decode" function in plms, so use "sample"
                            shape = [args.C, args.H // args.f, args.W // args.f]
                            samples, _ = sampler.sample(S=args.steps,
                                                            conditioning=c,
                                                            batch_size=args.n_samples,
                                                            shape=shape,
                                                            verbose=False,
                                                            unconditional_guidance_scale=args.scale,
                                                            unconditional_conditioning=uc,
                                                            eta=args.ddim_eta,
                                                            x_T=z_enc,
                                                            img_callback=callback)
                        else:
                            raise Exception(f"Sampler {args.sampler} not recognised.")

                    
                    if return_latent:
                        results.append(samples.clone())

                    x_samples = model.decode_first_stage(samples)

                    if args.use_mask and args.overlay_mask:
                        # Overlay the masked image after the image is generated
                        if args.init_sample is not None:
                            img_original = args.init_sample
                        elif init_image is not None:
                            img_original = init_image
                        else:
                            raise Exception("Cannot overlay the masked image without an init image to overlay")

                        mask_fullres = prepare_mask(args.mask_file if mask_image is None else mask_image, 
                                                    img_original.shape, 
                                                    args.mask_contrast_adjust, 
                                                    args.mask_brightness_adjust)
                        mask_fullres = mask_fullres[:,:3,:,:]
                        mask_fullres = repeat(mask_fullres, '1 ... -> b ...', b=batch_size)

                        mask_fullres[mask_fullres < mask_fullres.max()] = 0
                        mask_fullres = gaussian_filter(mask_fullres, args.mask_overlay_blur)
                        mask_fullres = torch.Tensor(mask_fullres).to(device)

                        x_samples = img_original * mask_fullres + x_samples * ((mask_fullres * -1.0) + 1)


                    if return_sample:
                        results.append(x_samples.clone())

                    x_samples = torch.clamp((x_samples + 1.0) / 2.0, min=0.0, max=1.0)

                    if return_c:
                        results.append(c.clone())

                    for x_sample in x_samples:
                        x_sample = 255. * rearrange(x_sample.cpu().numpy(), 'c h w -> h w c')
                        image = Image.fromarray(x_sample.astype(np.uint8))
                        results.append(image)
    return results

/home/jnl6052/music/.venv/lib64/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
#@markdown **Select and Load Model**

model_config = "v1-inference.yaml" #@param ["custom","v1-inference.yaml"]
model_checkpoint =  "custom" #@param ["custom","sd-v1-4-full-ema.ckpt","sd-v1-4.ckpt","sd-v1-3-full-ema.ckpt","sd-v1-3.ckpt","sd-v1-2-full-ema.ckpt","sd-v1-2.ckpt","sd-v1-1-full-ema.ckpt","sd-v1-1.ckpt", "robo-diffusion-v1.ckpt","waifu-diffusion-v1-3.ckpt"]
if model_checkpoint == "waifu-diffusion-v1-3.ckpt":
    model_checkpoint = "model-epoch05-float16.ckpt"
custom_config_path = "" #@param {type:"string"}
custom_checkpoint_path = selected_checkpoint #@param {type:"string"}

load_on_run_all = True #@param {type: 'boolean'}
half_precision = True # check
check_sha256 = True #@param {type:"boolean"}

model_map = {
    "sd-v1-4-full-ema.ckpt": {
        'sha256': '14749efc0ae8ef0329391ad4436feb781b402f4fece4883c7ad8d10556d8a36a',
        'url': 'https://huggingface.co/CompVis/stable-diffusion-v-1-2-original/blob/main/sd-v1-4-full-ema.ckpt',
        'requires_login': True,
        },
    "sd-v1-4.ckpt": {
        'sha256': 'fe4efff1e174c627256e44ec2991ba279b3816e364b49f9be2abc0b3ff3f8556',
        'url': 'https://huggingface.co/CompVis/stable-diffusion-v-1-4-original/resolve/main/sd-v1-4.ckpt',
        'requires_login': True,
        },
    "sd-v1-3-full-ema.ckpt": {
        'sha256': '54632c6e8a36eecae65e36cb0595fab314e1a1545a65209f24fde221a8d4b2ca',
        'url': 'https://huggingface.co/CompVis/stable-diffusion-v-1-3-original/blob/main/sd-v1-3-full-ema.ckpt',
        'requires_login': True,
        },
    "sd-v1-3.ckpt": {
        'sha256': '2cff93af4dcc07c3e03110205988ff98481e86539c51a8098d4f2236e41f7f2f',
        'url': 'https://huggingface.co/CompVis/stable-diffusion-v-1-3-original/resolve/main/sd-v1-3.ckpt',
        'requires_login': True,
        },
    "sd-v1-2-full-ema.ckpt": {
        'sha256': 'bc5086a904d7b9d13d2a7bccf38f089824755be7261c7399d92e555e1e9ac69a',
        'url': 'https://huggingface.co/CompVis/stable-diffusion-v-1-2-original/blob/main/sd-v1-2-full-ema.ckpt',
        'requires_login': True,
        },
    "sd-v1-2.ckpt": {
        'sha256': '3b87d30facd5bafca1cbed71cfb86648aad75d1c264663c0cc78c7aea8daec0d',
        'url': 'https://huggingface.co/CompVis/stable-diffusion-v-1-2-original/resolve/main/sd-v1-2.ckpt',
        'requires_login': True,
        },
    "sd-v1-1-full-ema.ckpt": {
        'sha256': 'efdeb5dc418a025d9a8cc0a8617e106c69044bc2925abecc8a254b2910d69829',
        'url':'https://huggingface.co/CompVis/stable-diffusion-v-1-1-original/resolve/main/sd-v1-1-full-ema.ckpt',
        'requires_login': True,
        },
    "sd-v1-1.ckpt": {
        'sha256': '86cd1d3ccb044d7ba8db743d717c9bac603c4043508ad2571383f954390f3cea',
        'url': 'https://huggingface.co/CompVis/stable-diffusion-v-1-1-original/resolve/main/sd-v1-1.ckpt',
        'requires_login': True,
        },
    "robo-diffusion-v1.ckpt": {
        'sha256': '244dbe0dcb55c761bde9c2ac0e9b46cc9705ebfe5f1f3a7cc46251573ea14e16',
        'url': 'https://huggingface.co/nousr/robo-diffusion/resolve/main/models/robo-diffusion-v1.ckpt',
        'requires_login': False,
        },
    "model-epoch05-float16.ckpt": {
        'sha256': '26cf2a2e30095926bb9fd9de0c83f47adc0b442dbfdc3d667d43778e8b70bece',
        'url': 'https://huggingface.co/hakurei/waifu-diffusion-v1-3/resolve/main/model-epoch05-float16.ckpt',
        'requires_login': False,
        },
}

# config path
ckpt_config_path = custom_config_path if model_config == "custom" else os.path.join(models_path, model_config)
if os.path.exists(ckpt_config_path):
    print(f"{ckpt_config_path} exists")
else:
    ckpt_config_path = "./stable-diffusion/configs/stable-diffusion/v1-inference.yaml"
print(f"Using config: {ckpt_config_path}")

# checkpoint path or download
ckpt_path = custom_checkpoint_path if model_checkpoint == "custom" else os.path.join(models_path, model_checkpoint)
ckpt_valid = True
if os.path.exists(ckpt_path):
    print(f"{ckpt_path} exists")
elif 'url' in model_map[model_checkpoint]:
    url = model_map[model_checkpoint]['url']

    # CLI dialogue to authenticate download
    if model_map[model_checkpoint]['requires_login']:
        print("This model requires an authentication token")
        print("Please ensure you have accepted its terms of service before continuing.")

        username = input("What is your huggingface username?:")
        token = input("What is your huggingface token?:")

        _, path = url.split("https://")

        url = f"https://{username}:{token}@{path}"

    # contact server for model
    print(f"Attempting to download {model_checkpoint}...this may take a while")
    ckpt_request = requests.get(url)
    request_status = ckpt_request.status_code

    # inform user of errors
    if request_status == 403:
      raise ConnectionRefusedError("You have not accepted the license for this model.")
    elif request_status == 404:
      raise ConnectionError("Could not make contact with server")
    elif request_status != 200:
      raise ConnectionError(f"Some other error has ocurred - response code: {request_status}")

    # write to model path
    with open(os.path.join(models_path, model_checkpoint), 'wb') as model_file:
        model_file.write(ckpt_request.content)
else:
    print(f"Please download model checkpoint and place in {os.path.join(models_path, model_checkpoint)}")
    ckpt_valid = False

if check_sha256 and model_checkpoint != "custom" and ckpt_valid:
    import hashlib
    print("\n...checking sha256")
    with open(ckpt_path, "rb") as f:
        bytes = f.read() 
        hash = hashlib.sha256(bytes).hexdigest()
        del bytes
    if model_map[model_checkpoint]["sha256"] == hash:
        print("hash is correct\n")
    else:
        print("hash in not correct\n")
        ckpt_valid = False

if ckpt_valid:
    print(f"Using ckpt: {ckpt_path}")

def load_model_from_config(config, ckpt, verbose=False, device='cuda', half_precision=True):
    map_location = "cuda" #@param ["cpu", "cuda"]
    print(f"Loading model from {ckpt}")
    pl_sd = torch.load(ckpt, map_location=map_location)
    if "global_step" in pl_sd:
        print(f"Global Step: {pl_sd['global_step']}")
    sd = pl_sd["state_dict"]
    model = instantiate_from_config(config.model)
    m, u = model.load_state_dict(sd, strict=False)
    if len(m) > 0 and verbose:
        print("missing keys:")
        print(m)
    if len(u) > 0 and verbose:
        print("unexpected keys:")
        print(u)

    if half_precision:
        model = model.half().to(device)
    else:
        model = model.to(device)
    model.eval()
    return model

if load_on_run_all and ckpt_valid:
    local_config = OmegaConf.load(f"{ckpt_config_path}")
    model = load_model_from_config(local_config, f"{ckpt_path}", half_precision=half_precision)
    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
    model = model.to(device)

Using config: ./stable-diffusion/configs/stable-diffusion/v1-inference.yaml
/home/jnl6052/music/model/sd-v1-4.ckpt exists
Using ckpt: /home/jnl6052/music/model/sd-v1-4.ckpt
Loading model from /home/jnl6052/music/model/sd-v1-4.ckpt
Global Step: 470000
LatentDiffusion: Running in eps-prediction mode
DiffusionWrapper has 859.52 M params.
making attention of type 'vanilla' with 512 in_channels
Working with z of shape (1, 4, 32, 32) = 4096 dimensions.
making attention of type 'vanilla' with 512 in_channels


Some weights of the model checkpoint at openai/clip-vit-large-patch14 were not used when initializing CLIPTextModel: ['vision_model.encoder.layers.10.mlp.fc1.weight', 'vision_model.embeddings.position_embedding.weight', 'vision_model.encoder.layers.15.self_attn.k_proj.weight', 'vision_model.encoder.layers.22.self_attn.q_proj.weight', 'vision_model.encoder.layers.19.self_attn.v_proj.weight', 'vision_model.encoder.layers.16.mlp.fc2.bias', 'vision_model.encoder.layers.8.self_attn.v_proj.bias', 'vision_model.encoder.layers.3.layer_norm1.bias', 'vision_model.encoder.layers.11.mlp.fc1.weight', 'vision_model.encoder.layers.4.layer_norm2.bias', 'vision_model.encoder.layers.3.self_attn.out_proj.bias', 'vision_model.encoder.layers.9.layer_norm1.bias', 'vision_model.encoder.layers.5.self_attn.out_proj.bias', 'vision_model.encoder.layers.16.mlp.fc1.weight', 'vision_model.encoder.layers.6.self_attn.out_proj.bias', 'vision_model.encoder.layers.15.self_attn.q_proj.weight', 'vision_model.encoder.layer

# Settings

### Animation Settings

In [7]:

def DeforumAnimArgs():

    #@markdown ####**Animation:**
    animation_mode = '2D' #@param ['None', '2D', '3D', 'Video Input', 'Interpolation'] {type:'string'}
    max_frames = 5000 #@param {type:"number"}
    border = 'replicate' #@param ['wrap', 'replicate'] {type:'string'}

    #@markdown ####**Motion Parameters:**
    angle = "0:(0)"#@param {type:"string"}
    zoom = "0: (1.00), 1: (1.00), 2: (1.00), 3: (1.01), 4: (1.01), 5: (1.06), 6: (1.01), 7: (1.00), 8: (1.00), 9: (1.00), 10: (1.00), 11: (1.00), 12: (1.00), 13: (1.00), 14: (1.00), 15: (1.00), 16: (1.00), 17: (1.00), 18: (1.00), 19: (1.00), 20: (1.00), 21: (1.00), 22: (1.00), 23: (1.00), 24: (1.00), 25: (1.00), 26: (1.00), 27: (1.00), 28: (1.00), 29: (1.00), 30: (1.00), 31: (1.00), 32: (1.01), 33: (1.04), 34: (1.03), 35: (1.04), 36: (1.07), 37: (1.03), 38: (1.00), 39: (1.00), 40: (1.00), 41: (1.11), 42: (1.07), 43: (1.01), 44: (1.00), 45: (1.00), 46: (1.00), 47: (1.00), 48: (1.00), 49: (1.00), 50: (1.00), 51: (1.00), 52: (1.00), 53: (1.00), 54: (1.00), 55: (1.00), 56: (1.00), 57: (1.00), 58: (1.00), 59: (1.00), 60: (1.00), 61: (1.00), 62: (1.00), 63: (1.00), 64: (1.00), 65: (1.00), 66: (1.00), 67: (1.00), 68: (1.00), 69: (1.00), 70: (1.00), 71: (1.10), 72: (1.00), 73: (1.00), 74: (1.00), 75: (1.00), 76: (1.00), 77: (1.00), 78: (1.00), 79: (1.00), 80: (1.00), 81: (1.00), 82: (1.00), 83: (1.00), 84: (1.00), 85: (1.00), 86: (1.00), 87: (1.00), 88: (1.00), 89: (1.00), 90: (1.00), 91: (1.00), 92: (1.00), 93: (1.00), 94: (1.16), 95: (1.06), 96: (1.07), 97: (1.04), 98: (1.01), 99: (1.21), 100: (1.05), 101: (1.17), 102: (1.00), 103: (1.00), 104: (1.00), 105: (1.00), 106: (1.00), 107: (1.00), 108: (1.00), 109: (1.00), 110: (1.00), 111: (1.00), 112: (1.00), 113: (1.00), 114: (1.21), 115: (1.07), 116: (1.09), 117: (1.00), 118: (1.00), 119: (1.00), 120: (1.00), 121: (1.00), 122: (1.00), 123: (1.00), 124: (1.00), 125: (1.00), 126: (1.00), 127: (1.00), 128: (1.15), 129: (1.00), 130: (1.00), 131: (1.00), 132: (1.00), 133: (1.00), 134: (1.00), 135: (1.00), 136: (1.00), 137: (1.00), 138: (1.00), 139: (1.00), 140: (1.00), 141: (1.00), 142: (1.00), 143: (1.00), 144: (1.00), 145: (1.00), 146: (1.00), 147: (1.00), 148: (1.00), 149: (1.00), 150: (1.00), 151: (1.00), 152: (1.00), 153: (1.00), 154: (1.00), 155: (1.00), 156: (1.16), 157: (1.11), 158: (1.02), 159: (1.00), 160: (1.00), 161: (1.00), 162: (1.00), 163: (1.00), 164: (1.00), 165: (1.00), 166: (1.00), 167: (1.00), 168: (1.00), 169: (1.00), 170: (1.00), 171: (1.00), 172: (1.00), 173: (1.00), 174: (1.00), 175: (1.00), 176: (1.00), 177: (1.00), 178: (1.00), 179: (1.00), 180: (1.00), 181: (1.00), 182: (1.00), 183: (1.00), 184: (1.00), 185: (1.06), 186: (1.02), 187: (1.00), 188: (1.00), 189: (1.00), 190: (1.00), 191: (1.00), 192: (1.00), 193: (1.00), 194: (1.00), 195: (1.00), 196: (1.00), 197: (1.00), 198: (1.00), 199: (1.00), 200: (1.00), 201: (1.00), 202: (1.00), 203: (1.00), 204: (1.00), 205: (1.00), 206: (1.00), 207: (1.00), 208: (1.00), 209: (1.00), 210: (1.00), 211: (1.00), 212: (1.00), 213: (1.03), 214: (1.14), 215: (1.14), 216: (1.02), 217: (1.00), 218: (1.00), 219: (1.00), 220: (1.00), 221: (1.00), 222: (1.00), 223: (1.00), 224: (1.00), 225: (1.00), 226: (1.00), 227: (1.00), 228: (1.14), 229: (1.08), 230: (1.04), 231: (1.00), 232: (1.00), 233: (1.00), 234: (1.00), 235: (1.00), 236: (1.00), 237: (1.00), 238: (1.00), 239: (1.00), 240: (1.00), 241: (1.00), 242: (1.09), 243: (1.00), 244: (1.00), 245: (1.00), 246: (1.00), 247: (1.00), 248: (1.00), 249: (1.00), 250: (1.00), 251: (1.00), 252: (1.00), 253: (1.00), 254: (1.00), 255: (1.00), 256: (1.00), 257: (1.00), 258: (1.00), 259: (1.00), 260: (1.00), 261: (1.00), 262: (1.00), 263: (1.00), 264: (1.11), 265: (1.08), 266: (1.07), 267: (1.02), 268: (1.01), 269: (1.00), 270: (1.04), 271: (1.10), 272: (1.04), 273: (1.00), 274: (1.00), 275: (1.00), 276: (1.00), 277: (1.00), 278: (1.00), 279: (1.00), 280: (1.00), 281: (1.00), 282: (1.00), 283: (1.00), 284: (1.00), 285: (1.16), 286: (1.04), 287: (1.05), 288: (1.00), 289: (1.00), 290: (1.00), 291: (1.00), 292: (1.00), 293: (1.00), 294: (1.00), 295: (1.00), 296: (1.00), 297: (1.00), 298: (1.00), 299: (1.11), 300: (1.00), 301: (1.00), 302: (1.00), 303: (1.00), 304: (1.00), 305: (1.00), 306: (1.00), 307: (1.00), 308: (1.00), 309: (1.00), 310: (1.00), 311: (1.00), 312: (1.00), 313: (1.00), 314: (1.00), 315: (1.00), 316: (1.00), 317: (1.00), 318: (1.00), 319: (1.00), 320: (1.00), 321: (1.00), 322: (1.00), 323: (1.00), 324: (1.00), 325: (1.00), 326: (1.00), 327: (1.15), 328: (1.09), 329: (1.07), 330: (1.01), 331: (1.00), 332: (1.00), 333: (1.00), 334: (1.00), 335: (1.00), 336: (1.00), 337: (1.00), 338: (1.00), 339: (1.00), 340: (1.00), 341: (1.00), 342: (1.00), 343: (1.00), 344: (1.00), 345: (1.00), 346: (1.00), 347: (1.00), 348: (1.00), 349: (1.00), 350: (1.00), 351: (1.00), 352: (1.00), 353: (1.00), 354: (1.00), 355: (1.00), 356: (1.14), 357: (1.00), 358: (1.00), 359: (1.00), 360: (1.00), 361: (1.00), 362: (1.00), 363: (1.00), 364: (1.00), 365: (1.00), 366: (1.00), 367: (1.00), 368: (1.00), 369: (1.00), 370: (1.00), 371: (1.00), 372: (1.00), 373: (1.00), 374: (1.00), 375: (1.00), 376: (1.00), 377: (1.00), 378: (1.11), 379: (1.09), 380: (1.06), 381: (1.02), 382: (1.01), 383: (1.00), 384: (1.11), 385: (1.10), 386: (1.02), 387: (1.01), 388: (1.00), 389: (1.00), 390: (1.00), 391: (1.00), 392: (1.00), 393: (1.00), 394: (1.00), 395: (1.00), 396: (1.00), 397: (1.00), 398: (1.00), 399: (1.00), 400: (1.00), 401: (1.00), 402: (1.00), 403: (1.00), 404: (1.00), 405: (1.00), 406: (1.00), 407: (1.00), 408: (1.00), 409: (1.00), 410: (1.00), 411: (1.00), 412: (1.00), 413: (1.05), 414: (1.00), 415: (1.00), 416: (1.00), 417: (1.00), 418: (1.00), 419: (1.00), 420: (1.00), 421: (1.00), 422: (1.00), 423: (1.00), 424: (1.00), 425: (1.00), 426: (1.00), 427: (1.00), 428: (1.00), 429: (1.00), 430: (1.00), 431: (1.00), 432: (1.00), 433: (1.00), 434: (1.00), 435: (1.00), 436: (1.00), 437: (1.00), 438: (1.00), 439: (1.00), 440: (1.02), 441: (1.09), 442: (1.14), 443: (1.01), 444: (1.00), 445: (1.00), 446: (1.00), 447: (1.00), 448: (1.00), 449: (1.00), 450: (1.00), 451: (1.00), 452: (1.00), 453: (1.00), 454: (1.00), 455: (1.13), 456: (1.09), 457: (1.10), 458: (1.00), 459: (1.00), 460: (1.00), 461: (1.00), 462: (1.00), 463: (1.00), 464: (1.00), 465: (1.00), 466: (1.00), 467: (1.00), 468: (1.01), 469: (1.03), 470: (1.00), 471: (1.00), 472: (1.00), 473: (1.00), 474: (1.00), 475: (1.00), 476: (1.12), 477: (1.05), 478: (1.07), 479: (1.00), 480: (1.00), 481: (1.00), 482: (1.00), 483: (1.00), 484: (1.00), 485: (1.00), 486: (1.00), 487: (1.00), 488: (1.00), 489: (1.00), 490: (1.00), 491: (1.00), 492: (1.00), 493: (1.00), 494: (1.00), 495: (1.15), 496: (1.13), 497: (1.02), 498: (1.00), 499: (1.00), 500: (1.00), 501: (1.00), 502: (1.00), 503: (1.00), 504: (1.00), 505: (1.00), 506: (1.00), 507: (1.00), 508: (1.00), 509: (1.00), 510: (1.00), 511: (1.00), 512: (1.00), 513: (1.00), 514: (1.00), 515: (1.00), 516: (1.00), 517: (1.00), 518: (1.00), 519: (1.00), 520: (1.00), 521: (1.00), 522: (1.00), 523: (1.00), 524: (1.00), 525: (1.14), 526: (1.00), 527: (1.00), 528: (1.00), 529: (1.00), 530: (1.00), 531: (1.00), 532: (1.00), 533: (1.00), 534: (1.00), 535: (1.00), 536: (1.00), 537: (1.00), 538: (1.00), 539: (1.00), 540: (1.00), 541: (1.00), 542: (1.00), 543: (1.00), 544: (1.00), 545: (1.00), 546: (1.00), 547: (1.00), 548: (1.00), 549: (1.00), 550: (1.00), 551: (1.00), 552: (1.01), 553: (1.12), 554: (1.07), 555: (1.03), 556: (1.00), 557: (1.00), 558: (1.00), 559: (1.00), 560: (1.00), 561: (1.00), 562: (1.00), 563: (1.00), 564: (1.00), 565: (1.00), 566: (1.00), 567: (1.05), 568: (1.09), 569: (1.14), 570: (1.00), 571: (1.00), 572: (1.00), 573: (1.00), 574: (1.00), 575: (1.00), 576: (1.00), 577: (1.00), 578: (1.00), 579: (1.00), 580: (1.00), 581: (1.14), 582: (1.00), 583: (1.00), 584: (1.00), 585: (1.00), 586: (1.00), 587: (1.00), 588: (1.00), 589: (1.00), 590: (1.00), 591: (1.00), 592: (1.00), 593: (1.00), 594: (1.00), 595: (1.00), 596: (1.00), 597: (1.00), 598: (1.00), 599: (1.00), 600: (1.00), 601: (1.00), 602: (1.00), 603: (1.00), 604: (1.00), 605: (1.00), 606: (1.00), 607: (1.00), 608: (1.00), 609: (1.00), 610: (1.13), 611: (1.09), 612: (1.02), 613: (1.00), 614: (1.00), 615: (1.00), 616: (1.00), 617: (1.00), 618: (1.00), 619: (1.00), 620: (1.00), 621: (1.00), 622: (1.00), 623: (1.00), 624: (1.00), 625: (1.00), 626: (1.00), 627: (1.00), 628: (1.00), 629: (1.00), 630: (1.00), 631: (1.00), 632: (1.00), 633: (1.00), 634: (1.00), 635: (1.00), 636: (1.00), 637: (1.00), 638: (1.01), 639: (1.03), 640: (1.00), 641: (1.00), 642: (1.00), 643: (1.00), 644: (1.00), 645: (1.00), 646: (1.00), 647: (1.00), 648: (1.00), 649: (1.00), 650: (1.00), 651: (1.00), 652: (1.00), 653: (1.00), 654: (1.00), 655: (1.00), 656: (1.00), 657: (1.00), 658: (1.00), 659: (1.00), 660: (1.00), 661: (1.00), 662: (1.00), 663: (1.00), 664: (1.00), 665: (1.00), 666: (1.01), 667: (1.12), 668: (1.11), 669: (1.02), 670: (1.00), 671: (1.00), 672: (1.00), 673: (1.00), 674: (1.00), 675: (1.00), 676: (1.00), 677: (1.00), 678: (1.00), 679: (1.00), 680: (1.00), 681: (1.08), 682: (1.06), 683: (1.10), 684: (1.00), 685: (1.00), 686: (1.00), 687: (1.00), 688: (1.00), 689: (1.00), 690: (1.00), 691: (1.00), 692: (1.00), 693: (1.00), 694: (1.00), 695: (1.05), 696: (1.01), 697: (1.00), 698: (1.00), 699: (1.00), 700: (1.00), 701: (1.00), 702: (1.00), 703: (1.00), 704: (1.00), 705: (1.00), 706: (1.00), 707: (1.00), 708: (1.00), 709: (1.00), 710: (1.00), 711: (1.00), 712: (1.00), 713: (1.00), 714: (1.00), 715: (1.00), 716: (1.00), 717: (1.00), 718: (1.00), 719: (1.00), 720: (1.00), 721: (1.00), 722: (1.00), 723: (1.06), 724: (1.11), 725: (1.06), 726: (1.01), 727: (1.00), 728: (1.00), 729: (1.00), 730: (1.00), 731: (1.00), 732: (1.00), 733: (1.00), 734: (1.00), 735: (1.00), 736: (1.00), 737: (1.00), 738: (1.00), 739: (1.00), 740: (1.00), 741: (1.00), 742: (1.00), 743: (1.00), 744: (1.00), 745: (1.00), 746: (1.00), 747: (1.00), 748: (1.00), 749: (1.00), 750: (1.00), 751: (1.00), 752: (1.11), 753: (1.00), 754: (1.00), 755: (1.00), 756: (1.00), 757: (1.00), 758: (1.00), 759: (1.00), 760: (1.00), 761: (1.00), 762: (1.00), 763: (1.00), 764: (1.00), 765: (1.00), 766: (1.00), 767: (1.00), 768: (1.00), 769: (1.00), 770: (1.00), 771: (1.00), 772: (1.00), 773: (1.00), 774: (1.00), 775: (1.20), 776: (1.05), 777: (1.05), 778: (1.01), 779: (1.06), 780: (1.10), 781: (1.19), 782: (1.00), 783: (1.00), 784: (1.00), 785: (1.00), 786: (1.00), 787: (1.00), 788: (1.00), 789: (1.00), 790: (1.00), 791: (1.00), 792: (1.00), 793: (1.00), 794: (1.23), 795: (1.07), 796: (1.02), 797: (1.00), 798: (1.00), 799: (1.00), 800: (1.00), 801: (1.00), 802: (1.00), 803: (1.00), 804: (1.00), 805: (1.00), 806: (1.00), 807: (1.02), 808: (1.05), 809: (1.00), 810: (1.00), 811: (1.00), 812: (1.00), 813: (1.00), 814: (1.00), 815: (1.00), 816: (1.00), 817: (1.00), 818: (1.00), 819: (1.00), 820: (1.00), 821: (1.00), 822: (1.00), 823: (1.00), 824: (1.00), 825: (1.00), 826: (1.00), 827: (1.00), 828: (1.00), 829: (1.00), 830: (1.00), 831: (1.00), 832: (1.00), 833: (1.00), 834: (1.00), 835: (1.02), 836: (1.14), 837: (1.17), 838: (1.02), 839: (1.00), 840: (1.00), 841: (1.00), 842: (1.00), 843: (1.00), 844: (1.00), 845: (1.00), 846: (1.00), 847: (1.00), 848: (1.00), 849: (1.00), 850: (1.00), 851: (1.00), 852: (1.00), 853: (1.00), 854: (1.00), 855: (1.00), 856: (1.00), 857: (1.00), 858: (1.00), 859: (1.00), 860: (1.00), 861: (1.00), 862: (1.00), 863: (1.00), 864: (1.11), 865: (1.01), 866: (1.00), 867: (1.00), 868: (1.00), 869: (1.00), 870: (1.00), 871: (1.00), 872: (1.00), 873: (1.00), 874: (1.00), 875: (1.00), 876: (1.00), 877: (1.00), 878: (1.00), 879: (1.00), 880: (1.00), 881: (1.00), 882: (1.00), 883: (1.00), 884: (1.00), 885: (1.00), 886: (1.08), 887: (1.12), 888: (1.06), 889: (1.01), 890: (1.00), 891: (1.05), 892: (1.11), 893: (1.21), 894: (1.00), 895: (1.00), 896: (1.00), 897: (1.00), 898: (1.00), 899: (1.00), 900: (1.00), 901: (1.00), 902: (1.00), 903: (1.00), 904: (1.00), 905: (1.02), 906: (1.21), 907: (1.15), 908: (1.02), 909: (1.00), 910: (1.00), 911: (1.00), 912: (1.00), 913: (1.00), 914: (1.00), 915: (1.00), 916: (1.00), 917: (1.00), 918: (1.00), 919: (1.00), 920: (1.00), 921: (1.00), 922: (1.00), 923: (1.00), 924: (1.00), 925: (1.00), 926: (1.02), 927: (1.06), 928: (1.01), 929: (1.00), 930: (1.00), 931: (1.00), 932: (1.00), 933: (1.08), 934: (1.00), 935: (1.00), 936: (1.00), 937: (1.00), 938: (1.00), 939: (1.00), 940: (1.00), 941: (1.00), 942: (1.00), 943: (1.00), 944: (1.00), 945: (1.00), 946: (1.00), 947: (1.17), 948: (1.03), 949: (1.01), 950: (1.00), 951: (1.00), 952: (1.00), 953: (1.00), 954: (1.00), 955: (1.00), 956: (1.00), 957: (1.00), 958: (1.00), 959: (1.00), 960: (1.00), 961: (1.00), 962: (1.00), 963: (1.00), 964: (1.00), 965: (1.00), 966: (1.00), 967: (1.00), 968: (1.00), 969: (1.00), 970: (1.00), 971: (1.00), 972: (1.00), 973: (1.00), 974: (1.00), 975: (1.02), 976: (1.15), 977: (1.01), 978: (1.00), 979: (1.00), 980: (1.00), 981: (1.00), 982: (1.00), 983: (1.00), 984: (1.00), 985: (1.00), 986: (1.00), 987: (1.00), 988: (1.00), 989: (1.00), 990: (1.00), 991: (1.00), 992: (1.00), 993: (1.00), 994: (1.00), 995: (1.00), 996: (1.00), 997: (1.07), 998: (1.15), 999: (1.06), 1000: (1.01), 1001: (1.00), 1002: (1.01), 1003: (1.18), 1004: (1.08), 1005: (1.03), 1006: (1.01), 1007: (1.00), 1008: (1.00), 1009: (1.00), 1010: (1.00), 1011: (1.00), 1012: (1.00), 1013: (1.00), 1014: (1.00), 1015: (1.00), 1016: (1.00), 1017: (1.28), 1018: (1.18), 1019: (1.02), 1020: (1.00), 1021: (1.00), 1022: (1.00), 1023: (1.00), 1024: (1.00), 1025: (1.00), 1026: (1.00), 1027: (1.00), 1028: (1.00), 1029: (1.00), 1030: (1.01), 1031: (1.14), 1032: (1.00), 1033: (1.00), 1034: (1.00), 1035: (1.00), 1036: (1.00), 1037: (1.00), 1038: (1.00), 1039: (1.00), 1040: (1.00), 1041: (1.00), 1042: (1.00), 1043: (1.00), 1044: (1.00), 1045: (1.00), 1046: (1.00), 1047: (1.00), 1048: (1.00), 1049: (1.00), 1050: (1.00), 1051: (1.00), 1052: (1.00), 1053: (1.00), 1054: (1.00), 1055: (1.00), 1056: (1.00), 1057: (1.00), 1058: (1.01), 1059: (1.14), 1060: (1.09), 1061: (1.01), 1062: (1.00), 1063: (1.00), 1064: (1.00), 1065: (1.00), 1066: (1.00), 1067: (1.00), 1068: (1.00), 1069: (1.00), 1070: (1.00), 1071: (1.00), 1072: (1.00), 1073: (1.00), 1074: (1.00), 1075: (1.00), 1076: (1.00), 1077: (1.00), 1078: (1.00), 1079: (1.00), 1080: (1.00), 1081: (1.00), 1082: (1.00), 1083: (1.00), 1084: (1.00), 1085: (1.00), 1086: (1.00), 1087: (1.01), 1088: (1.08), 1089: (1.00), 1090: (1.00), 1091: (1.00), 1092: (1.00), 1093: (1.00), 1094: (1.00), 1095: (1.00), 1096: (1.00), 1097: (1.00), 1098: (1.00), 1099: (1.00), 1100: (1.00), 1101: (1.00), 1102: (1.00), 1103: (1.00), 1104: (1.00), 1105: (1.00), 1106: (1.00), 1107: (1.00), 1108: (1.00), 1109: (1.00), 1110: (1.00), 1111: (1.00), 1112: (1.00), 1113: (1.00), 1114: (1.00), 1115: (1.22), 1116: (1.09), 1117: (1.02), 1118: (1.00), 1119: (1.00), 1120: (1.00), 1121: (1.00), 1122: (1.00), 1123: (1.00), 1124: (1.00), 1125: (1.00), 1126: (1.00), 1127: (1.00), 1128: (1.00), 1129: (1.14), 1130: (1.17), 1131: (1.03), 1132: (1.01), 1133: (1.00), 1134: (1.00), 1135: (1.00), 1136: (1.00), 1137: (1.00), 1138: (1.00), 1139: (1.00), 1140: (1.00), 1141: (1.00), 1142: (1.00), 1143: (1.28), 1144: (1.01), 1145: (1.00), 1146: (1.00), 1147: (1.00), 1148: (1.00), 1149: (1.00), 1150: (1.00), 1151: (1.00), 1152: (1.00), 1153: (1.00), 1154: (1.00), 1155: (1.00), 1156: (1.00), 1157: (1.00), 1158: (1.00), 1159: (1.00), 1160: (1.00), 1161: (1.00), 1162: (1.00), 1163: (1.00), 1164: (1.00), 1165: (1.00), 1166: (1.00), 1167: (1.00), 1168: (1.00), 1169: (1.00), 1170: (1.03), 1171: (1.11), 1172: (1.09), 1173: (1.01), 1174: (1.00), 1175: (1.00), 1176: (1.00), 1177: (1.00), 1178: (1.00), 1179: (1.00), 1180: (1.00), 1181: (1.00), 1182: (1.00), 1183: (1.00), 1184: (1.00), 1185: (1.24), 1186: (1.20), 1187: (1.03), 1188: (1.00), 1189: (1.00), 1190: (1.00), 1191: (1.00), 1192: (1.00), 1193: (1.00), 1194: (1.00), 1195: (1.00), 1196: (1.00), 1197: (1.00), 1198: (1.00), 1199: (1.17), 1200: (1.00), 1201: (1.00), 1202: (1.00), 1203: (1.00), 1204: (1.00), 1205: (1.00), 1206: (1.00), 1207: (1.00), 1208: (1.00), 1209: (1.00), 1210: (1.00), 1211: (1.00), 1212: (1.00), 1213: (1.00), 1214: (1.00), 1215: (1.00), 1216: (1.00), 1217: (1.00), 1218: (1.00), 1219: (1.00), 1220: (1.00), 1221: (1.00), 1222: (1.00), 1223: (1.00), 1224: (1.00), 1225: (1.00), 1226: (1.22), 1227: (1.16), 1228: (1.02), 1229: (1.00), 1230: (1.00), 1231: (1.00), 1232: (1.00), 1233: (1.00), 1234: (1.00), 1235: (1.00), 1236: (1.00), 1237: (1.00), 1238: (1.00), 1239: (1.00), 1240: (1.03), 1241: (1.13), 1242: (1.12), 1243: (1.00), 1244: (1.00), 1245: (1.00), 1246: (1.00), 1247: (1.00), 1248: (1.00), 1249: (1.00), 1250: (1.00), 1251: (1.00), 1252: (1.00), 1253: (1.00), 1254: (1.12), 1255: (1.00), 1256: (1.00), 1257: (1.00), 1258: (1.00), 1259: (1.00), 1260: (1.00), 1261: (1.01), 1262: (1.22), 1263: (1.08), 1264: (1.01), 1265: (1.00), 1266: (1.00), 1267: (1.00), 1268: (1.00), 1269: (1.00), 1270: (1.00), 1271: (1.00), 1272: (1.00), 1273: (1.00), 1274: (1.00), 1275: (1.00), 1276: (1.00), 1277: (1.00), 1278: (1.00), 1279: (1.00), 1280: (1.17), 1281: (1.16), 1282: (1.01), 1283: (1.00), 1284: (1.00), 1285: (1.00), 1286: (1.00), 1287: (1.00), 1288: (1.00), 1289: (1.00), 1290: (1.00), 1291: (1.00), 1292: (1.00), 1293: (1.00), 1294: (1.00), 1295: (1.00), 1296: (1.00), 1297: (1.00), 1298: (1.00), 1299: (1.00), 1300: (1.00), 1301: (1.00), 1302: (1.00), 1303: (1.00), 1304: (1.00), 1305: (1.00), 1306: (1.00), 1307: (1.00), 1308: (1.00), 1309: (1.12), 1310: (1.01), 1311: (1.00), 1312: (1.00), 1313: (1.00), 1314: (1.00), 1315: (1.00), 1316: (1.00), 1317: (1.00), 1318: (1.00), 1319: (1.00), 1320: (1.00), 1321: (1.00), 1322: (1.00), 1323: (1.00), 1324: (1.00), 1325: (1.00), 1326: (1.00), 1327: (1.00), 1328: (1.00), 1329: (1.00), 1330: (1.00), 1331: (1.00), 1332: (1.00), 1333: (1.00), 1334: (1.00), 1335: (1.00), 1336: (1.00), 1337: (1.13), 1338: (1.11), 1339: (1.03), 1340: (1.00), 1341: (1.00), 1342: (1.00), 1343: (1.00), 1344: (1.00), 1345: (1.00), 1346: (1.00), 1347: (1.00), 1348: (1.00), 1349: (1.00), 1350: (1.00), 1351: (1.21), 1352: (1.09), 1353: (1.04), 1354: (1.00), 1355: (1.00), 1356: (1.00), 1357: (1.00), 1358: (1.00), 1359: (1.00), 1360: (1.00), 1361: (1.00), 1362: (1.00), 1363: (1.00), 1364: (1.00), 1365: (1.28), 1366: (1.00), 1367: (1.00), 1368: (1.00), 1369: (1.00), 1370: (1.00), 1371: (1.00), 1372: (1.00), 1373: (1.00), 1374: (1.00), 1375: (1.00), 1376: (1.00), 1377: (1.00), 1378: (1.00), 1379: (1.00), 1380: (1.00), 1381: (1.00), 1382: (1.00), 1383: (1.00), 1384: (1.00), 1385: (1.00), 1386: (1.00), 1387: (1.00), 1388: (1.00), 1389: (1.00), 1390: (1.00), 1391: (1.00), 1392: (1.05), 1393: (1.16), 1394: (1.05), 1395: (1.01), 1396: (1.00), 1397: (1.00), 1398: (1.00), 1399: (1.00), 1400: (1.00), 1401: (1.00), 1402: (1.00), 1403: (1.00), 1404: (1.00), 1405: (1.00), 1406: (1.00), 1407: (1.00), 1408: (1.00), 1409: (1.00), 1410: (1.00), 1411: (1.00), 1412: (1.00), 1413: (1.00), 1414: (1.00), 1415: (1.00), 1416: (1.00), 1417: (1.00), 1418: (1.00), 1419: (1.00), 1420: (1.00), 1421: (1.00), 1422: (1.27), 1423: (1.00), 1424: (1.00), 1425: (1.00), 1426: (1.00), 1427: (1.00), 1428: (1.00), 1429: (1.00), 1430: (1.00), 1431: (1.00), 1432: (1.00), 1433: (1.00), 1434: (1.00), 1435: (1.00), 1436: (1.00), 1437: (1.00), 1438: (1.00), 1439: (1.00), 1440: (1.00), 1441: (1.00), 1442: (1.00), 1443: (1.00), 1444: (1.00), 1445: (1.00), 1446: (1.00), 1447: (1.00), 1448: (1.00), 1449: (1.15), 1450: (1.09), 1451: (1.05), 1452: (1.00), 1453: (1.00), 1454: (1.00), 1455: (1.00), 1456: (1.00), 1457: (1.00), 1458: (1.00), 1459: (1.00), 1460: (1.00), 1461: (1.00), 1462: (1.00), 1463: (1.04), 1464: (1.13), 1465: (1.08), 1466: (1.00), 1467: (1.00), 1468: (1.00), 1469: (1.00), 1470: (1.00), 1471: (1.00), 1472: (1.00), 1473: (1.00), 1474: (1.00), 1475: (1.00), 1476: (1.00), 1477: (1.13), 1478: (1.02), 1479: (1.00), 1480: (1.00), 1481: (1.00), 1482: (1.00), 1483: (1.00), 1484: (1.00), 1485: (1.00), 1486: (1.00), 1487: (1.00), 1488: (1.00), 1489: (1.00), 1490: (1.00), 1491: (1.00), 1492: (1.00), 1493: (1.00), 1494: (1.00), 1495: (1.00), 1496: (1.00), 1497: (1.00), 1498: (1.00), 1499: (1.00), 1500: (1.00), 1501: (1.00), 1502: (1.00), 1503: (1.00), 1504: (1.00), 1505: (1.13), 1506: (1.12), 1507: (1.05), 1508: (1.01), 1509: (1.00), 1510: (1.00), 1511: (1.00), 1512: (1.00), 1513: (1.00), 1514: (1.00), 1515: (1.00), 1516: (1.00), 1517: (1.00), 1518: (1.00), 1519: (1.00), 1520: (1.00), 1521: (1.00), 1522: (1.00), 1523: (1.00), 1524: (1.00), 1525: (1.00), 1526: (1.00), 1527: (1.00), 1528: (1.00), 1529: (1.00), 1530: (1.00), 1531: (1.00), 1532: (1.00), 1533: (1.00), 1534: (1.17), 1535: (1.01), 1536: (1.00), 1537: (1.00), 1538: (1.00), 1539: (1.00), 1540: (1.00), 1541: (1.00), 1542: (1.00), 1543: (1.00), 1544: (1.00), 1545: (1.00), 1546: (1.00), 1547: (1.00), 1548: (1.00), 1549: (1.00), 1550: (1.00), 1551: (1.00), 1552: (1.00), 1553: (1.00), 1554: (1.00), 1555: (1.00), 1556: (1.00), 1557: (1.00), 1558: (1.00), 1559: (1.00), 1560: (1.00), 1561: (1.00), 1562: (1.16), 1563: (1.08), 1564: (1.01), 1565: (1.00), 1566: (1.00), 1567: (1.00), 1568: (1.00), 1569: (1.00), 1570: (1.00), 1571: (1.00), 1572: (1.00), 1573: (1.00), 1574: (1.00), 1575: (1.00), 1576: (1.15), 1577: (1.08), 1578: (1.04), 1579: (1.00), 1580: (1.00), 1581: (1.00), 1582: (1.00), 1583: (1.00), 1584: (1.00), 1585: (1.00), 1586: (1.00), 1587: (1.00), 1588: (1.00), 1589: (1.00), 1590: (1.10), 1591: (1.00), 1592: (1.00), 1593: (1.00), 1594: (1.00), 1595: (1.00), 1596: (1.00), 1597: (1.00), 1598: (1.00), 1599: (1.00), 1600: (1.00), 1601: (1.00), 1602: (1.00), 1603: (1.00), 1604: (1.00), 1605: (1.00), 1606: (1.00), 1607: (1.00), 1608: (1.00), 1609: (1.00), 1610: (1.00), 1611: (1.00), 1612: (1.00), 1613: (1.00), 1614: (1.00), 1615: (1.00), 1616: (1.00), 1617: (1.00), 1618: (1.22), 1619: (1.09), 1620: (1.03), 1621: (1.00), 1622: (1.00), 1623: (1.00), 1624: (1.00), 1625: (1.00), 1626: (1.00), 1627: (1.00), 1628: (1.00), 1629: (1.00), 1630: (1.00), 1631: (1.00), 1632: (1.00), 1633: (1.00), 1634: (1.00), 1635: (1.00), 1636: (1.00), 1637: (1.00), 1638: (1.00), 1639: (1.00), 1640: (1.00), 1641: (1.00), 1642: (1.00), 1643: (1.00), 1644: (1.00), 1645: (1.00), 1646: (1.00), 1647: (1.10), 1648: (1.00), 1649: (1.00), 1650: (1.00), 1651: (1.00), 1652: (1.00), 1653: (1.00), 1654: (1.00), 1655: (1.00), 1656: (1.00), 1657: (1.00), 1658: (1.00), 1659: (1.00), 1660: (1.00), 1661: (1.00), 1662: (1.00), 1663: (1.00), 1664: (1.00), 1665: (1.00), 1666: (1.00), 1667: (1.00), 1668: (1.00), 1669: (1.00), 1670: (1.00), 1671: (1.00), 1672: (1.00), 1673: (1.00), 1674: (1.05), 1675: (1.11), 1676: (1.14), 1677: (1.00), 1678: (1.00), 1679: (1.00), 1680: (1.00), 1681: (1.00), 1682: (1.00), 1683: (1.00), 1684: (1.00), 1685: (1.00), 1686: (1.00), 1687: (1.00), 1688: (1.00), 1689: (1.26), 1690: (1.10), 1691: (1.09), 1692: (1.00), 1693: (1.00), 1694: (1.00), 1695: (1.00), 1696: (1.00), 1697: (1.00), 1698: (1.00), 1699: (1.00), 1700: (1.00), 1701: (1.00), 1702: (1.01), 1703: (1.12), 1704: (1.00), 1705: (1.00), 1706: (1.00), 1707: (1.00), 1708: (1.00), 1709: (1.00), 1710: (1.00), 1711: (1.00), 1712: (1.00), 1713: (1.00), 1714: (1.00), 1715: (1.00), 1716: (1.00), 1717: (1.00), 1718: (1.00), 1719: (1.00), 1720: (1.00), 1721: (1.00), 1722: (1.00), 1723: (1.00), 1724: (1.00), 1725: (1.00), 1726: (1.00), 1727: (1.00), 1728: (1.00), 1729: (1.00), 1730: (1.01), 1731: (1.19), 1732: (1.20), 1733: (1.02), 1734: (1.00), 1735: (1.00), 1736: (1.00), 1737: (1.00), 1738: (1.00), 1739: (1.00), 1740: (1.00), 1741: (1.00), 1742: (1.00), 1743: (1.00), 1744: (1.00), 1745: (1.00), 1746: (1.00), 1747: (1.00), 1748: (1.00), 1749: (1.00), 1750: (1.00), 1751: (1.00), 1752: (1.00), 1753: (1.00), 1754: (1.00), 1755: (1.00), 1756: (1.00), 1757: (1.00), 1758: (1.00), 1759: (1.19), 1760: (1.01), 1761: (1.00), 1762: (1.00), 1763: (1.00), 1764: (1.00), 1765: (1.00), 1766: (1.00), 1767: (1.00), 1768: (1.00), 1769: (1.00), 1770: (1.00), 1771: (1.00), 1772: (1.00), 1773: (1.00), 1774: (1.00), 1775: (1.00), 1776: (1.00), 1777: (1.00), 1778: (1.00), 1779: (1.00), 1780: (1.00), 1781: (1.02), 1782: (1.00), 1783: (1.00), 1784: (1.00), 1785: (1.06), 1786: (2.00), 1787: (1.31), 1788: (1.28), 1789: (1.12), 1790: (1.04), 1791: (1.02), 1792: (1.00), 1793: (1.00), 1794: (1.00), 1795: (1.14), 1796: (1.02), 1797: (1.00), 1798: (1.00), 1799: (1.00), 1800: (1.00), 1801: (1.00), 1802: (1.00), 1803: (1.00), 1804: (1.00), 1805: (1.00), 1806: (1.00), 1807: (1.00), 1808: (1.04), 1809: (1.16), 1810: (1.05), 1811: (1.03), 1812: (1.01), 1813: (1.01), 1814: (1.00), 1815: (1.00), 1816: (1.00), 1817: (1.00), 1818: (1.00), 1819: (1.00), 1820: (1.00), 1821: (1.00), 1822: (1.25), 1823: (1.04), 1824: (1.01), 1825: (1.00), 1826: (1.00), 1827: (1.09), 1828: (1.06), 1829: (1.01), 1830: (1.00), 1831: (1.00), 1832: (1.00), 1833: (1.00), 1834: (1.00), 1835: (1.00), 1836: (1.00), 1837: (1.00), 1838: (1.00), 1839: (1.00), 1840: (1.00), 1841: (1.16), 1842: (1.11), 1843: (1.01), 1844: (1.00), 1845: (1.00), 1846: (1.00), 1847: (1.00), 1848: (1.00), 1849: (1.00), 1850: (1.00), 1851: (1.00), 1852: (1.00), 1853: (1.00), 1854: (1.00), 1855: (1.00), 1856: (1.00), 1857: (1.00), 1858: (1.00), 1859: (1.00), 1860: (1.00), 1861: (1.00), 1862: (1.00), 1863: (1.00), 1864: (1.00), 1865: (1.00), 1866: (1.00), 1867: (1.00), 1868: (1.00), 1869: (1.00), 1870: (1.00), 1871: (1.05), 1872: (1.01), 1873: (1.00), 1874: (1.00), 1875: (1.00), 1876: (1.00), 1877: (1.00), 1878: (1.00), 1879: (1.00), 1880: (1.00), 1881: (1.00), 1882: (1.00), 1883: (1.00), 1884: (1.00), 1885: (1.00), 1886: (1.00), 1887: (1.00), 1888: (1.00), 1889: (1.00), 1890: (1.00), 1891: (1.00), 1892: (1.00), 1893: (1.01), 1894: (1.16), 1895: (1.06), 1896: (1.03), 1897: (1.00), 1898: (1.00), 1899: (1.16), 1900: (1.04), 1901: (1.02), 1902: (1.00), 1903: (1.00), 1904: (1.00), 1905: (1.00), 1906: (1.00), 1907: (1.00), 1908: (1.00), 1909: (1.00), 1910: (1.00), 1911: (1.00), 1912: (1.00), 1913: (1.14), 1914: (1.06), 1915: (1.04), 1916: (1.00), 1917: (1.00), 1918: (1.00), 1919: (1.00), 1920: (1.00), 1921: (1.00), 1922: (1.00), 1923: (1.00), 1924: (1.00), 1925: (1.00), 1926: (1.00), 1927: (1.00), 1928: (1.00), 1929: (1.00), 1930: (1.00), 1931: (1.00), 1932: (1.00), 1933: (1.00), 1934: (1.00), 1935: (1.09), 1936: (1.00), 1937: (1.00), 1938: (1.00), 1939: (1.00), 1940: (1.00), 1941: (1.01), 1942: (1.32), 1943: (1.14), 1944: (1.08), 1945: (1.03), 1946: (1.01), 1947: (1.00), 1948: (1.00), 1949: (1.00), 1950: (1.00), 1951: (1.00), 1952: (1.00), 1953: (1.00), 1954: (1.00), 1955: (1.00), 1956: (1.26), 1957: (1.08), 1958: (1.02), 1959: (1.00), 1960: (1.00), 1961: (1.00), 1962: (1.00), 1963: (1.00), 1964: (1.00), 1965: (1.00), 1966: (1.00), 1967: (1.00), 1968: (1.00), 1969: (1.00), 1970: (1.00), 1971: (1.00), 1972: (1.00), 1973: (1.00), 1974: (1.00), 1975: (1.00), 1976: (1.00), 1977: (1.00), 1978: (1.00), 1979: (1.00), 1980: (1.00), 1981: (1.00), 1982: (1.00), 1983: (1.00), 1984: (1.00), 1985: (1.16), 1986: (1.00), 1987: (1.00), 1988: (1.00), 1989: (1.00), 1990: (1.00), 1991: (1.00), 1992: (1.00), 1993: (1.00), 1994: (1.00), 1995: (1.00), 1996: (1.00), 1997: (1.00), 1998: (1.00), 1999: (1.00), 2000: (1.00), 2001: (1.00), 2002: (1.00), 2003: (1.00), 2004: (1.00), 2005: (1.00), 2006: (1.00), 2007: (1.01), 2008: (1.00), 2009: (1.00), 2010: (1.00), 2011: (1.00), 2012: (1.16), 2013: (1.22), 2014: (1.11), 2015: (1.03), 2016: (1.00), 2017: (1.00), 2018: (1.00), 2019: (1.00), 2020: (1.00), 2021: (1.00), 2022: (1.00), 2023: (1.00), 2024: (1.00), 2025: (1.00), 2026: (1.00), 2027: (1.00), 2028: (1.00), 2029: (1.00), 2030: (1.00), 2031: (1.00), 2032: (1.00), 2033: (1.00), 2034: (1.00), 2035: (1.00), 2036: (1.00), 2037: (1.00), 2038: (1.00), 2039: (1.00), 2040: (1.55), 2041: (1.17), 2042: (1.21), 2043: (1.06), 2044: (1.02), 2045: (1.01), 2046: (1.00), 2047: (1.01), 2048: (1.10), 2049: (1.02), 2050: (1.01), 2051: (1.00), 2052: (1.00), 2053: (1.00), 2054: (1.00), 2055: (1.00), 2056: (1.00), 2057: (1.00), 2058: (1.00), 2059: (1.00), 2060: (1.00), 2061: (1.00), 2062: (1.00), 2063: (1.00), 2064: (1.00), 2065: (1.00), 2066: (1.00), 2067: (1.21), 2068: (1.09), 2069: (1.01), 2070: (1.00), 2071: (1.00), 2072: (1.00), 2073: (1.00), 2074: (1.00), 2075: (1.00), 2076: (1.00), 2077: (1.00), 2078: (1.00), 2079: (1.00), 2080: (1.00), 2081: (1.00), 2082: (1.00), 2083: (1.00), 2084: (1.00), 2085: (1.00), 2086: (1.00), 2087: (1.00), 2088: (1.00), 2089: (1.00), 2090: (1.00), 2091: (1.00), 2092: (1.00), 2093: (1.00), 2094: (1.00), 2095: (1.00), 2096: (1.13), 2097: (1.00), 2098: (1.00), 2099: (1.00), 2100: (1.00), 2101: (1.00), 2102: (1.00), 2103: (1.00), 2104: (1.00), 2105: (1.00), 2106: (1.00), 2107: (1.00), 2108: (1.00), 2109: (1.00), 2110: (1.00), 2111: (1.00), 2112: (1.00), 2113: (1.00), 2114: (1.00), 2115: (1.00), 2116: (1.00), 2117: (1.00), 2118: (1.00), 2119: (1.17), 2120: (1.05), 2121: (1.03), 2122: (1.00), 2123: (1.08), 2124: (1.07), 2125: (1.08), 2126: (1.00), 2127: (1.00), 2128: (1.00), 2129: (1.00), 2130: (1.00), 2131: (1.00), 2132: (1.00), 2133: (1.00), 2134: (1.00), 2135: (1.00), 2136: (1.00), 2137: (1.00), 2138: (1.23), 2139: (1.04), 2140: (1.03), 2141: (1.00), 2142: (1.00), 2143: (1.00), 2144: (1.00), 2145: (1.00), 2146: (1.00), 2147: (1.00), 2148: (1.00), 2149: (1.00), 2150: (1.00), 2151: (1.15), 2152: (1.00), 2153: (1.00), 2154: (1.00), 2155: (1.00), 2156: (1.00), 2157: (1.00), 2158: (1.00), 2159: (1.11), 2160: (1.00), 2161: (1.00), 2162: (1.00), 2163: (1.00), 2164: (1.00), 2165: (1.00), 2166: (1.00), 2167: (1.00), 2168: (1.00), 2169: (1.00), 2170: (1.00), 2171: (1.00), 2172: (1.00), 2173: (1.00), 2174: (1.00), 2175: (1.00), 2176: (1.00), 2177: (1.00), 2178: (1.15), 2179: (1.07), 2180: (1.03), 2181: (1.00), 2182: (1.00), 2183: (1.00), 2184: (1.00), 2185: (1.00), 2186: (1.00), 2187: (1.00), 2188: (1.00), 2189: (1.00), 2190: (1.00), 2191: (1.00), 2192: (1.00), 2193: (1.00), 2194: (1.00), 2195: (1.00), 2196: (1.00), 2197: (1.00), 2198: (1.00), 2199: (1.00), 2200: (1.00), 2201: (1.00), 2202: (1.00), 2203: (1.00), 2204: (1.00), 2205: (1.00), 2206: (1.00), 2207: (1.03), 2208: (1.00), 2209: (1.00), 2210: (1.00), 2211: (1.00), 2212: (1.00), 2213: (1.00), 2214: (1.00), 2215: (1.00), 2216: (1.00), 2217: (1.00), 2218: (1.00), 2219: (1.00), 2220: (1.00), 2221: (1.00), 2222: (1.00), 2223: (1.00), 2224: (1.00), 2225: (1.00), 2226: (1.00), 2227: (1.00), 2228: (1.00), 2229: (1.00), 2230: (1.00), 2231: (1.00), 2232: (1.00), 2233: (1.00), 2234: (1.00), 2235: (1.00), 2236: (1.10), 2237: (1.05), 2238: (1.02), 2239: (1.00), 2240: (1.00), 2241: (1.00), 2242: (1.00), 2243: (1.00), 2244: (1.00), 2245: (1.00), 2246: (1.00), 2247: (1.00), 2248: (1.00), 2249: (1.00), 2250: (1.00), 2251: (1.00), 2252: (1.00), 2253: (1.00), 2254: (1.00), 2255: (1.00), 2256: (1.00), 2257: (1.00), 2258: (1.00), 2259: (1.00), 2260: (1.00), 2261: (1.00), 2262: (1.00), 2263: (1.00), 2264: (1.03), 2265: (1.00), 2266: (1.00), 2267: (1.00), 2268: (1.00), 2269: (1.00), 2270: (1.00), 2271: (1.00), 2272: (1.00), 2273: (1.00), 2274: (1.00), 2275: (1.00), 2276: (1.00), 2277: (1.00), 2278: (1.00), 2279: (1.00), 2280: (1.00), 2281: (1.00), 2282: (1.00), 2283: (1.00), 2284: (1.00), 2285: (1.00), 2286: (1.00), 2287: (1.00), 2288: (1.00), 2289: (1.00), 2290: (1.00), 2291: (1.00), 2292: (1.00), 2293: (1.09), 2294: (1.10), 2295: (1.02), 2296: (1.00), 2297: (1.00), 2298: (1.00), 2299: (1.00), 2300: (1.00), 2301: (1.00), 2302: (1.00), 2303: (1.00), 2304: (1.00), 2305: (1.00), 2306: (1.00), 2307: (1.00), 2308: (1.00), 2309: (1.00), 2310: (1.00), 2311: (1.00), 2312: (1.00), 2313: (1.00), 2314: (1.00), 2315: (1.00), 2316: (1.00), 2317: (1.00), 2318: (1.00), 2319: (1.00), 2320: (1.00), 2321: (1.00), 2322: (1.16), 2323: (1.01), 2324: (1.00), 2325: (1.00), 2326: (1.00), 2327: (1.00), 2328: (1.00), 2329: (1.00), 2330: (1.00), 2331: (1.00), 2332: (1.00), 2333: (1.00), 2334: (1.00), 2335: (1.00), 2336: (1.00), 2337: (1.00), 2338: (1.00), 2339: (1.00), 2340: (1.00), 2341: (1.00), 2342: (1.00), 2343: (1.00), 2344: (1.00), 2345: (1.04), 2346: (1.12), 2347: (1.04), 2348: (1.01), 2349: (1.00), 2350: (1.10), 2351: (1.15), 2352: (1.12), 2353: (1.00), 2354: (1.00), 2355: (1.00), 2356: (1.00), 2357: (1.00), 2358: (1.00), 2359: (1.00), 2360: (1.00), 2361: (1.00), 2362: (1.00), 2363: (1.00), 2364: (1.00), 2365: (1.17), 2366: (1.07), 2367: (1.07), 2368: (1.00), 2369: (1.00), 2370: (1.00), 2371: (1.00), 2372: (1.00), 2373: (1.00), 2374: (1.00), 2375: (1.00), 2376: (1.00), 2377: (1.00), 2378: (1.00), 2379: (1.21), 2380: (1.00), 2381: (1.00), 2382: (1.00), 2383: (1.00), 2384: (1.00), 2385: (1.00), 2386: (1.00), 2387: (1.00), 2388: (1.00), 2389: (1.00), 2390: (1.00), 2391: (1.00), 2392: (1.09), 2393: (1.01), 2394: (1.00), 2395: (1.00), 2396: (1.00), 2397: (1.00), 2398: (1.00), 2399: (1.00), 2400: (1.00), 2401: (1.00), 2402: (1.00), 2403: (1.00), 2404: (1.00), 2405: (1.00), 2406: (1.02), 2407: (1.10), 2408: (1.04), 2409: (1.01), 2410: (1.00), 2411: (1.00), 2412: (1.00), 2413: (1.00), 2414: (1.00), 2415: (1.00), 2416: (1.00), 2417: (1.00), 2418: (1.00), 2419: (1.00), 2420: (1.00), 2421: (1.00), 2422: (1.00), 2423: (1.00), 2424: (1.00), 2425: (1.00), 2426: (1.00), 2427: (1.00), 2428: (1.00), 2429: (1.00), 2430: (1.00), 2431: (1.00), 2432: (1.00), 2433: (1.00), 2434: (1.00), 2435: (1.17), 2436: (1.01), 2437: (1.00), 2438: (1.00), 2439: (1.00), 2440: (1.00), 2441: (1.00), 2442: (1.00), 2443: (1.00), 2444: (1.00), 2445: (1.00), 2446: (1.00), 2447: (1.00), 2448: (1.00), 2449: (1.00), 2450: (1.00), 2451: (1.00), 2452: (1.00), 2453: (1.00), 2454: (1.00), 2455: (1.00), 2456: (1.00), 2457: (1.00), 2458: (1.27), 2459: (1.05), 2460: (1.04), 2461: (1.01), 2462: (1.07), 2463: (1.10), 2464: (1.12), 2465: (1.00), 2466: (1.00), 2467: (1.00), 2468: (1.00), 2469: (1.00), 2470: (1.00), 2471: (1.00), 2472: (1.00), 2473: (1.00), 2474: (1.00), 2475: (1.00), 2476: (1.03), 2477: (1.20), 2478: (1.16), 2479: (1.02), 2480: (1.00), 2481: (1.00), 2482: (1.00), 2483: (1.00), 2484: (1.00), 2485: (1.00), 2486: (1.00), 2487: (1.00), 2488: (1.00), 2489: (1.00), 2490: (1.13), 2491: (1.01), 2492: (1.00), 2493: (1.00), 2494: (1.00), 2495: (1.00), 2496: (1.00), 2497: (1.00), 2498: (1.06), 2499: (1.16), 2500: (1.05), 2501: (1.01), 2502: (1.00), 2503: (1.00), 2504: (1.00), 2505: (1.00), 2506: (1.00), 2507: (1.00), 2508: (1.00), 2509: (1.00), 2510: (1.00), 2511: (1.00), 2512: (1.00), 2513: (1.00), 2514: (1.00), 2515: (1.00), 2516: (1.00), 2517: (1.27), 2518: (1.13), 2519: (1.01), 2520: (1.00), 2521: (1.00), 2522: (1.00), 2523: (1.00), 2524: (1.00), 2525: (1.00), 2526: (1.00), 2527: (1.00), 2528: (1.00), 2529: (1.00), 2530: (1.00), 2531: (1.00), 2532: (1.00), 2533: (1.00), 2534: (1.00), 2535: (1.00), 2536: (1.00), 2537: (1.00), 2538: (1.00), 2539: (1.00), 2540: (1.00), 2541: (1.00), 2542: (1.00), 2543: (1.00), 2544: (1.00), 2545: (1.00), 2546: (1.00), 2547: (1.18), 2548: (1.00), 2549: (1.00), 2550: (1.00), 2551: (1.00), 2552: (1.00), 2553: (1.00), 2554: (1.00), 2555: (1.00), 2556: (1.00), 2557: (1.00), 2558: (1.00), 2559: (1.00), 2560: (1.00), 2561: (1.00), 2562: (1.00), 2563: (1.00), 2564: (1.00), 2565: (1.00), 2566: (1.00), 2567: (1.00), 2568: (1.00), 2569: (1.00), 2570: (1.00), 2571: (1.00), 2572: (1.00), 2573: (1.00), 2574: (1.02), 2575: (1.29), 2576: (1.03), 2577: (1.27), 2578: (1.01), 2579: (1.01), 2580: (1.00), 2581: (1.00), 2582: (1.00), 2583: (1.00), 2584: (1.00), 2585: (1.00), 2586: (1.00), 2587: (1.00), 2588: (1.00), 2589: (1.01), 2590: (1.14), 2591: (1.17), 2592: (1.01), 2593: (1.00), 2594: (1.00), 2595: (1.00), 2596: (1.00), 2597: (1.00), 2598: (1.00), 2599: (1.00), 2600: (1.00), 2601: (1.00), 2602: (1.00), 2603: (1.00), 2604: (1.10), 2605: (1.00), 2606: (1.00), 2607: (1.00), 2608: (1.00), 2609: (1.00), 2610: (1.00), 2611: (1.00), 2612: (1.00), 2613: (1.00), 2614: (1.00), 2615: (1.00), 2616: (1.00), 2617: (1.00), 2618: (1.00), 2619: (1.00), 2620: (1.00), 2621: (1.00), 2622: (1.00), 2623: (1.00), 2624: (1.00), 2625: (1.00), 2626: (1.00), 2627: (1.00), 2628: (1.00), 2629: (1.00), 2630: (1.00), 2631: (1.02), 2632: (1.13), 2633: (1.07), 2634: (1.02), 2635: (1.00), 2636: (1.00), 2637: (1.00), 2638: (1.00), 2639: (1.00), 2640: (1.00), 2641: (1.00), 2642: (1.00), 2643: (1.00), 2644: (1.00), 2645: (1.00), 2646: (1.00), 2647: (1.00), 2648: (1.00), 2649: (1.00), 2650: (1.00), 2651: (1.00), 2652: (1.00), 2653: (1.00), 2654: (1.00), 2655: (1.00), 2656: (1.00), 2657: (1.00), 2658: (1.00), 2659: (1.00), 2660: (1.19), 2661: (1.00), 2662: (1.00), 2663: (1.00), 2664: (1.00), 2665: (1.00), 2666: (1.00), 2667: (1.00), 2668: (1.00), 2669: (1.00), 2670: (1.00), 2671: (1.00), 2672: (1.00), 2673: (1.00), 2674: (1.00), 2675: (1.00), 2676: (1.00), 2677: (1.00), 2678: (1.00), 2679: (1.00), 2680: (1.00), 2681: (1.00), 2682: (1.00), 2683: (1.00), 2684: (1.00), 2685: (1.00), 2686: (1.00), 2687: (1.01), 2688: (1.14), 2689: (1.10), 2690: (1.00), 2691: (1.00), 2692: (1.00), 2693: (1.00), 2694: (1.00), 2695: (1.05), 2696: (1.10), 2697: (1.13), 2698: (1.00), 2699: (1.00), 2700: (1.00), 2701: (1.00), 2702: (1.00), 2703: (1.00), 2704: (1.00), 2705: (1.00), 2706: (1.00), 2707: (1.00), 2708: (1.00), 2709: (1.00), 2710: (1.00), 2711: (1.00), 2712: (1.00), 2713: (1.00), 2714: (1.02), 2715: (1.06), 2716: (1.00), 2717: (1.00), 2718: (1.00), 2719: (1.00), 2720: (1.00), 2721: (1.00), 2722: (1.00), 2723: (1.00), 2724: (1.00), 2725: (1.00), 2726: (1.00), 2727: (1.00), 2728: (1.00), 2729: (1.00), 2730: (1.00), 2731: (1.00), 2732: (1.00), 2733: (1.00), 2734: (1.00), 2735: (1.00), 2736: (1.00), 2737: (1.00), 2738: (1.00), 2739: (1.00), 2740: (1.00), 2741: (1.00), 2742: (1.00), 2743: (1.07), 2744: (1.12), 2745: (1.01), 2746: (1.00), 2747: (1.00), 2748: (1.00), 2749: (1.00), 2750: (1.00), 2751: (1.00), 2752: (1.00), 2753: (1.00), 2754: (1.00), 2755: (1.00), 2756: (1.00), 2757: (1.00), 2758: (1.00), 2759: (1.00), 2760: (1.00), 2761: (1.00), 2762: (1.00), 2763: (1.00), 2764: (1.00), 2765: (1.00), 2766: (1.00), 2767: (1.00), 2768: (1.00), 2769: (1.00), 2770: (1.00), 2771: (1.00), 2772: (1.15), 2773: (1.00), 2774: (1.00), 2775: (1.00), 2776: (1.00), 2777: (1.00), 2778: (1.00), 2779: (1.00), 2780: (1.00), 2781: (1.00), 2782: (1.00), 2783: (1.00), 2784: (1.00), 2785: (1.00), 2786: (1.00), 2787: (1.00), 2788: (1.00), 2789: (1.00), 2790: (1.00), 2791: (1.00), 2792: (1.00), 2793: (1.00), 2794: (1.00), 2795: (1.00), 2796: (1.00), 2797: (1.00), 2798: (1.00), 2799: (1.00), 2800: (1.11), 2801: (1.08), 2802: (1.08), 2803: (1.00), 2804: (1.00), 2805: (1.00), 2806: (1.00), 2807: (1.00), 2808: (1.00), 2809: (1.00), 2810: (1.00), 2811: (1.00), 2812: (1.00), 2813: (1.00), 2814: (1.11), 2815: (1.11), 2816: (1.11), 2817: (1.00), 2818: (1.00), 2819: (1.00), 2820: (1.00), 2821: (1.00), 2822: (1.00), 2823: (1.00), 2824: (1.00), 2825: (1.00), 2826: (1.00), 2827: (1.00), 2828: (1.06), 2829: (1.02), 2830: (1.00), 2831: (1.00), 2832: (1.00), 2833: (1.00), 2834: (1.00), 2835: (1.00), 2836: (1.00), 2837: (1.00), 2838: (1.00), 2839: (1.00), 2840: (1.00), 2841: (1.00), 2842: (1.00), 2843: (1.00), 2844: (1.00), 2845: (1.00), 2846: (1.00), 2847: (1.00), 2848: (1.00), 2849: (1.00), 2850: (1.00), 2851: (1.00), 2852: (1.00), 2853: (1.00), 2854: (1.00), 2855: (1.00), 2856: (1.06), 2857: (1.14), 2858: (1.03), 2859: (1.01), 2860: (1.00), 2861: (1.00), 2862: (1.00), 2863: (1.00), 2864: (1.00), 2865: (1.00), 2866: (1.00), 2867: (1.00), 2868: (1.00), 2869: (1.00), 2870: (1.00), 2871: (1.00), 2872: (1.00), 2873: (1.00), 2874: (1.00), 2875: (1.00), 2876: (1.00), 2877: (1.00), 2878: (1.00), 2879: (1.00), 2880: (1.00), 2881: (1.00), 2882: (1.00), 2883: (1.00), 2884: (1.00), 2885: (1.24), 2886: (1.00), 2887: (1.00), 2888: (1.00), 2889: (1.00), 2890: (1.00), 2891: (1.00), 2892: (1.00), 2893: (1.00), 2894: (1.00), 2895: (1.00), 2896: (1.00), 2897: (1.00), 2898: (1.00), 2899: (1.00), 2900: (1.00), 2901: (1.00), 2902: (1.00), 2903: (1.00), 2904: (1.00), 2905: (1.00), 2906: (1.00), 2907: (1.00), 2908: (1.00), 2909: (1.00), 2910: (1.00), 2911: (1.00), 2912: (1.00), 2913: (1.12), 2914: (1.04), 2915: (1.02), 2916: (1.00), 2917: (1.00), 2918: (1.00), 2919: (1.00), 2920: (1.00), 2921: (1.00), 2922: (1.00), 2923: (1.00), 2924: (1.00), 2925: (1.00), 2926: (1.00), 2927: (1.15), 2928: (1.07), 2929: (1.04), 2930: (1.00), 2931: (1.00), 2932: (1.00), 2933: (1.00), 2934: (1.00), 2935: (1.00), 2936: (1.00), 2937: (1.00), 2938: (1.00), 2939: (1.00), 2940: (1.00), 2941: (1.00), 2942: (1.00), 2943: (1.00), 2944: (1.00), 2945: (1.00), 2946: (1.00), 2947: (1.00), 2948: (1.00), 2949: (1.08), 2950: (1.00), 2951: (1.00), 2952: (1.00), 2953: (1.00), 2954: (1.00), 2955: (1.00), 2956: (1.00), 2957: (1.00), 2958: (1.00), 2959: (1.00), 2960: (1.00), 2961: (1.00), 2962: (1.00), 2963: (1.00), 2964: (1.00), 2965: (1.00), 2966: (1.00), 2967: (1.00), 2968: (1.03), 2969: (1.13), 2970: (1.03), 2971: (1.00), 2972: (1.00), 2973: (1.00), 2974: (1.00), 2975: (1.00), 2976: (1.00), 2977: (1.00), 2978: (1.00), 2979: (1.00), 2980: (1.00), 2981: (1.00), 2982: (1.00), 2983: (1.00), 2984: (1.00), 2985: (1.00), 2986: (1.00), 2987: (1.00), 2988: (1.00), 2989: (1.00), 2990: (1.00), 2991: (1.00), 2992: (1.00), 2993: (1.00), 2994: (1.00), 2995: (1.00), 2996: (1.00), 2997: (1.23), 2998: (1.00), 2999: (1.00), 3000: (1.00), 3001: (1.00), 3002: (1.00), 3003: (1.00), 3004: (1.00), 3005: (1.00), 3006: (1.00), 3007: (1.00), 3008: (1.00), 3009: (1.00), 3010: (1.00), 3011: (1.00), 3012: (1.00), 3013: (1.00), 3014: (1.00), 3015: (1.00), 3016: (1.00), 3017: (1.00), 3018: (1.00), 3019: (1.00), 3020: (1.00), 3021: (1.00), 3022: (1.00), 3023: (1.00), 3024: (1.00), 3025: (1.19), 3026: (1.05), 3027: (1.01), 3028: (1.00), 3029: (1.00), 3030: (1.00), 3031: (1.00), 3032: (1.00), 3033: (1.00), 3034: (1.00), 3035: (1.00), 3036: (1.00), 3037: (1.00), 3038: (1.00), 3039: (1.07), 3040: (1.11), 3041: (1.08), 3042: (1.01), 3043: (1.00), 3044: (1.00), 3045: (1.00), 3046: (1.00), 3047: (1.00), 3048: (1.00), 3049: (1.00), 3050: (1.00), 3051: (1.00), 3052: (1.00), 3053: (1.00), 3054: (1.19), 3055: (1.00), 3056: (1.00), 3057: (1.00), 3058: (1.00), 3059: (1.00), 3060: (1.00), 3061: (1.00), 3062: (1.00), 3063: (1.00), 3064: (1.00), 3065: (1.00), 3066: (1.00), 3067: (1.00), 3068: (1.00), 3069: (1.00), 3070: (1.00), 3071: (1.00), 3072: (1.00), 3073: (1.00), 3074: (1.00), 3075: (1.00), 3076: (1.00), 3077: (1.00), 3078: (1.00), 3079: (1.00), 3080: (1.00), 3081: (1.12), 3082: (1.10), 3083: (1.02), 3084: (1.00), 3085: (1.00), 3086: (1.00), 3087: (1.00), 3088: (1.00), 3089: (1.00), 3090: (1.00), 3091: (1.00), 3092: (1.00), 3093: (1.00), 3094: (1.00), 3095: (1.00), 3096: (1.00), 3097: (1.00), 3098: (1.00), 3099: (1.00), 3100: (1.00), 3101: (1.00), 3102: (1.00), 3103: (1.00), 3104: (1.00), 3105: (1.00), 3106: (1.00), 3107: (1.00), 3108: (1.00), 3109: (1.00), 3110: (1.19), 3111: (1.00), 3112: (1.00), 3113: (1.00), 3114: (1.00), 3115: (1.00), 3116: (1.00), 3117: (1.00), 3118: (1.00), 3119: (1.00), 3120: (1.00), 3121: (1.00), 3122: (1.00), 3123: (1.00), 3124: (1.00), 3125: (1.00), 3126: (1.00), 3127: (1.00), 3128: (1.00), 3129: (1.00), 3130: (1.00), 3131: (1.00), 3132: (1.00), 3133: (1.00), 3134: (1.00), 3135: (1.00), 3136: (1.00), 3137: (1.00), 3138: (1.00), 3139: (1.20), 3140: (1.10), 3141: (1.02), 3142: (1.00), 3143: (1.00), 3144: (1.00), 3145: (1.00), 3146: (1.00), 3147: (1.00), 3148: (1.00), 3149: (1.00), 3150: (1.00), 3151: (1.00), 3152: (1.00), 3153: (1.21), 3154: (1.06), 3155: (1.03), 3156: (1.00), 3157: (1.00), 3158: (1.00), 3159: (1.00), 3160: (1.00), 3161: (1.00), 3162: (1.00), 3163: (1.00), 3164: (1.00), 3165: (1.00), 3166: (1.00), 3167: (1.25), 3168: (1.01), 3169: (1.00), 3170: (1.00), 3171: (1.00), 3172: (1.00), 3173: (1.00), 3174: (1.00), 3175: (1.05), 3176: (1.01), 3177: (1.00), 3178: (1.00), 3179: (1.00), 3180: (1.00), 3181: (1.00), 3182: (1.00), 3183: (1.00), 3184: (1.00), 3185: (1.00), 3186: (1.00), 3187: (1.00), 3188: (1.00), 3189: (1.00), 3190: (1.00), 3191: (1.00), 3192: (1.00), 3193: (1.00), 3194: (1.17), 3195: (1.14), 3196: (1.01), 3197: (1.00), 3198: (1.00), 3199: (1.00), 3200: (1.00), 3201: (1.00), 3202: (1.00), 3203: (1.00), 3204: (1.00), 3205: (1.00), 3206: (1.00), 3207: (1.00), 3208: (1.00), 3209: (1.00), 3210: (1.00), 3211: (1.00), 3212: (1.00), 3213: (1.00), 3214: (1.00), 3215: (1.00), 3216: (1.00), 3217: (1.00), 3218: (1.00), 3219: (1.00), 3220: (1.00), 3221: (1.00), 3222: (1.00), 3223: (1.01), 3224: (1.01), 3225: (1.00), 3226: (1.00), 3227: (1.00), 3228: (1.00), 3229: (1.00), 3230: (1.00), 3231: (1.00), 3232: (1.00), 3233: (1.00), 3234: (1.00), 3235: (1.00), 3236: (1.00), 3237: (1.00), 3238: (1.00), 3239: (1.00), 3240: (1.00), 3241: (1.00), 3242: (1.00), 3243: (1.00), 3244: (1.00), 3245: (1.00), 3246: (1.00), 3247: (1.23), 3248: (1.04), 3249: (1.03), 3250: (1.01), 3251: (1.01), 3252: (1.16), 3253: (1.04), 3254: (1.03), 3255: (1.00), 3256: (1.00), 3257: (1.00), 3258: (1.00), 3259: (1.00), 3260: (1.00), 3261: (1.00), 3262: (1.00), 3263: (1.00), 3264: (1.00), 3265: (1.00), 3266: (1.06), 3267: (1.09), 3268: (1.09), 3269: (1.00), 3270: (1.00), 3271: (1.00), 3272: (1.00), 3273: (1.00), 3274: (1.00), 3275: (1.00), 3276: (1.00), 3277: (1.00), 3278: (1.00), 3279: (1.00), 3280: (1.09), 3281: (1.00), 3282: (1.00), 3283: (1.00), 3284: (1.00), 3285: (1.00), 3286: (1.00), 3287: (1.00), 3288: (1.28), 3289: (1.20), 3290: (1.15), 3291: (1.05), 3292: (1.03), 3293: (1.01), 3294: (1.00), 3295: (1.00), 3296: (1.00), 3297: (1.00), 3298: (1.00), 3299: (1.00), 3300: (1.00), 3301: (1.00), 3302: (1.00), 3303: (1.00), 3304: (1.00), 3305: (1.00), 3306: (1.00), 3307: (1.02), 3308: (1.18), 3309: (1.12), 3310: (1.01), 3311: (1.00), 3312: (1.00), 3313: (1.00), 3314: (1.00), 3315: (1.00), 3316: (1.00), 3317: (1.00), 3318: (1.00), 3319: (1.00), 3320: (1.00), 3321: (1.00), 3322: (1.00), 3323: (1.00), 3324: (1.00), 3325: (1.00), 3326: (1.00), 3327: (1.00), 3328: (1.00), 3329: (1.00), 3330: (1.00), 3331: (1.00), 3332: (1.00), 3333: (1.00), 3334: (1.00), 3335: (1.00), 3336: (1.00), 3337: (1.01), 3338: (1.01), 3339: (1.00), 3340: (1.00), 3341: (1.00), 3342: (1.00), 3343: (1.00), 3344: (1.00), 3345: (1.00), 3346: (1.00), 3347: (1.00), 3348: (1.00), 3349: (1.00), 3350: (1.00), 3351: (1.00), 3352: (1.00), 3353: (1.00), 3354: (1.00), 3355: (1.00), 3356: (1.00), 3357: (1.00), 3358: (1.00), 3359: (1.00), 3360: (1.00), 3361: (1.20), 3362: (1.06), 3363: (1.02), 3364: (1.00), 3365: (1.10), 3366: (1.07), 3367: (1.10), 3368: (1.00), 3369: (1.00), 3370: (1.00), 3371: (1.00), 3372: (1.00), 3373: (1.00), 3374: (1.00), 3375: (1.00), 3376: (1.00), 3377: (1.00), 3378: (1.00), 3379: (1.13), 3380: (1.04), 3381: (1.09), 3382: (1.00), 3383: (1.00), 3384: (1.00), 3385: (1.00), 3386: (1.00), 3387: (1.00), 3388: (1.00), 3389: (1.00), 3390: (1.00), 3391: (1.00), 3392: (1.00), 3393: (1.00), 3394: (1.00), 3395: (1.00), 3396: (1.00), 3397: (1.00), 3398: (1.00), 3399: (1.00), 3400: (1.06), 3401: (1.00), 3402: (1.00), 3403: (1.00), 3404: (1.00), 3405: (1.00), 3406: (1.01), 3407: (1.05), 3408: (1.01), 3409: (1.00), 3410: (1.00), 3411: (1.00), 3412: (1.00), 3413: (1.00), 3414: (1.00), 3415: (1.00), 3416: (1.00), 3417: (1.00), 3418: (1.00), 3419: (1.00), 3420: (1.01), 3421: (1.13), 3422: (1.08), 3423: (1.00), 3424: (1.00), 3425: (1.00), 3426: (1.00), 3427: (1.00), 3428: (1.00), 3429: (1.00), 3430: (1.00), 3431: (1.00), 3432: (1.00), 3433: (1.00), 3434: (1.00), 3435: (1.00), 3436: (1.00), 3437: (1.00), 3438: (1.00), 3439: (1.00), 3440: (1.00), 3441: (1.00), 3442: (1.00), 3443: (1.00), 3444: (1.00), 3445: (1.00), 3446: (1.00), 3447: (1.00), 3448: (1.00), 3449: (1.01), 3450: (1.02), 3451: (1.00), 3452: (1.00), 3453: (1.00), 3454: (1.00), 3455: (1.00), 3456: (1.00), 3457: (1.00), 3458: (1.00), 3459: (1.00), 3460: (1.00), 3461: (1.00), 3462: (1.00), 3463: (1.00), 3464: (1.00), 3465: (1.00), 3466: (1.00), 3467: (1.00), 3468: (1.00), 3469: (1.00), 3470: (1.00), 3471: (1.00), 3472: (1.01), 3473: (1.16), 3474: (1.04), 3475: (1.02), 3476: (1.00), 3477: (1.17), 3478: (1.04), 3479: (1.10), 3480: (1.00), 3481: (1.00), 3482: (1.00), 3483: (1.00), 3484: (1.00), 3485: (1.00), 3486: (1.00), 3487: (1.00), 3488: (1.00), 3489: (1.00), 3490: (1.00), 3491: (1.08), 3492: (1.10), 3493: (1.08), 3494: (1.00), 3495: (1.00), 3496: (1.00), 3497: (1.00), 3498: (1.00), 3499: (1.00), 3500: (1.00), 3501: (1.00), 3502: (1.00), 3503: (1.00), 3504: (1.00), 3505: (1.13), 3506: (1.00), 3507: (1.00), 3508: (1.00), 3509: (1.00), 3510: (1.00), 3511: (1.00), 3512: (1.00), 3513: (1.08), 3514: (1.28), 3515: (1.17), 3516: (1.07), 3517: (1.04), 3518: (1.01), 3519: (1.00), 3520: (1.00), 3521: (1.00), 3522: (1.00), 3523: (1.00), 3524: (1.00), 3525: (1.00), 3526: (1.00), 3527: (1.00), 3528: (1.00), 3529: (1.00), 3530: (1.00), 3531: (1.00), 3532: (1.00), 3533: (1.14), 3534: (1.06), 3535: (1.05), 3536: (1.00), 3537: (1.00), 3538: (1.00), 3539: (1.00), 3540: (1.00), 3541: (1.00), 3542: (1.00), 3543: (1.00), 3544: (1.00), 3545: (1.00), 3546: (1.00), 3547: (1.00), 3548: (1.00), 3549: (1.00), 3550: (1.00), 3551: (1.00), 3552: (1.00), 3553: (1.00), 3554: (1.00), 3555: (1.00), 3556: (1.00), 3557: (1.00), 3558: (1.00), 3559: (1.00), 3560: (1.00), 3561: (1.00), 3562: (1.09), 3563: (1.00), 3564: (1.00), 3565: (1.00), 3566: (1.00), 3567: (1.00), 3568: (1.00), 3569: (1.00), 3570: (1.00), 3571: (1.00), 3572: (1.00), 3573: (1.00), 3574: (1.00), 3575: (1.00), 3576: (1.00), 3577: (1.00), 3578: (1.00), 3579: (1.00), 3580: (1.00), 3581: (1.00), 3582: (1.00), 3583: (1.00), 3584: (1.00), 3585: (1.00), 3586: (1.00), 3587: (1.00), 3588: (1.00), 3589: (1.00), 3590: (1.00), 3591: (1.17), 3592: (1.06), 3593: (1.02), 3594: (1.00), 3595: (1.00), 3596: (1.00), 3597: (1.00), 3598: (1.00), 3599: (1.00), 3600: (1.00), 3601: (1.00), 3602: (1.00), 3603: (1.00), 3604: (1.00), 3605: (1.00), 3606: (1.00), 3607: (1.00), 3608: (1.00), 3609: (1.00), 3610: (1.00), 3611: (1.00), 3612: (1.00), 3613: (1.00), 3614: (1.00), 3615: (1.00), 3616: (1.00), 3617: (1.00), 3618: (1.00), 3619: (1.07), 3620: (1.00), 3621: (1.00), 3622: (1.00), 3623: (1.00), 3624: (1.00), 3625: (1.00), 3626: (1.00), 3627: (1.00), 3628: (1.00), 3629: (1.00), 3630: (1.00), 3631: (1.00), 3632: (1.00), 3633: (1.00), 3634: (1.00), 3635: (1.00), 3636: (1.00), 3637: (1.00), 3638: (1.00), 3639: (1.00), 3640: (1.00), 3641: (1.00), 3642: (1.00), 3643: (1.00), 3644: (1.00), 3645: (1.00), 3646: (1.00), 3647: (1.13), 3648: (1.17), 3649: (1.00), 3650: (1.00), 3651: (1.00), 3652: (1.00), 3653: (1.00), 3654: (1.00), 3655: (1.00), 3656: (1.00), 3657: (1.00), 3658: (1.00), 3659: (1.00), 3660: (1.00), 3661: (1.00), 3662: (1.00), 3663: (1.00), 3664: (1.00), 3665: (1.00), 3666: (1.00), 3667: (1.00), 3668: (1.00), 3669: (1.00), 3670: (1.00), 3671: (1.00), 3672: (1.00), 3673: (1.00), 3674: (1.00), 3675: (1.00), 3676: (1.20), 3677: (1.01), 3678: (1.00), 3679: (1.00), 3680: (1.00), 3681: (1.00), 3682: (1.00), 3683: (1.00), 3684: (1.00), 3685: (1.00), 3686: (1.00), 3687: (1.00), 3688: (1.00), 3689: (1.00), 3690: (1.00), 3691: (1.00), 3692: (1.00), 3693: (1.00), 3694: (1.00), 3695: (1.00), 3696: (1.00), 3697: (1.00), 3698: (1.00), 3699: (1.19), 3700: (1.04), 3701: (1.02), 3702: (1.01), 3703: (1.04), 3704: (1.09), 3705: (1.17), 3706: (1.00), 3707: (1.00), 3708: (1.00), 3709: (1.00), 3710: (1.00), 3711: (1.00), 3712: (1.00), 3713: (1.00), 3714: (1.00), 3715: (1.00), 3716: (1.00), 3717: (1.10), 3718: (1.19), 3719: (1.05), 3720: (1.00), 3721: (1.00), 3722: (1.00), 3723: (1.00), 3724: (1.00), 3725: (1.00), 3726: (1.00), 3727: (1.00), 3728: (1.00), 3729: (1.00), 3730: (1.00), 3731: (1.20), 3732: (1.00), 3733: (1.00), 3734: (1.00), 3735: (1.00), 3736: (1.00), 3737: (1.00), 3738: (1.00), 3739: (1.18), 3740: (1.10), 3741: (1.01), 3742: (1.00), 3743: (1.00), 3744: (1.00), 3745: (1.00), 3746: (1.00), 3747: (1.00), 3748: (1.00), 3749: (1.00), 3750: (1.00), 3751: (1.00), 3752: (1.00), 3753: (1.00), 3754: (1.00), 3755: (1.00), 3756: (1.00), 3757: (1.14), 3758: (1.19), 3759: (1.01), 3760: (1.00), 3761: (1.00), 3762: (1.00), 3763: (1.00), 3764: (1.00), 3765: (1.00), 3766: (1.00), 3767: (1.00), 3768: (1.00), 3769: (1.00), 3770: (1.00), 3771: (1.00), 3772: (1.00), 3773: (1.00), 3774: (1.00), 3775: (1.00), 3776: (1.00), 3777: (1.00), 3778: (1.00), 3779: (1.00), 3780: (1.00), 3781: (1.00), 3782: (1.00), 3783: (1.00), 3784: (1.00), 3785: (1.08), 3786: (1.02), 3787: (1.00), 3788: (1.00), 3789: (1.00), 3790: (1.00), 3791: (1.00), 3792: (1.00), 3793: (1.00), 3794: (1.00), 3795: (1.00), 3796: (1.00), 3797: (1.00), 3798: (1.00), 3799: (1.00), 3800: (1.00), 3801: (1.00), 3802: (1.00), 3803: (1.00), 3804: (1.00), 3805: (1.00), 3806: (1.00), 3807: (1.18), 3808: (1.05), 3809: (1.03), 3810: (1.00), 3811: (1.02), 3812: (1.16), 3813: (1.22), 3814: (1.01), 3815: (1.00), 3816: (1.00), 3817: (1.00), 3818: (1.00), 3819: (1.00), 3820: (1.00), 3821: (1.00), 3822: (1.00), 3823: (1.00), 3824: (1.00), 3825: (1.00), 3826: (1.04), 3827: (1.27), 3828: (1.12), 3829: (1.00), 3830: (1.00), 3831: (1.00), 3832: (1.00), 3833: (1.00), 3834: (1.00), 3835: (1.00), 3836: (1.00), 3837: (1.00), 3838: (1.00), 3839: (1.00), 3840: (1.00), 3841: (1.00), 3842: (1.00), 3843: (1.00), 3844: (1.00), 3845: (1.00), 3846: (1.00), 3847: (1.00), 3848: (1.02), 3849: (1.03), 3850: (1.00), 3851: (1.00), 3852: (1.00), 3853: (1.00), 3854: (1.00), 3855: (1.00), 3856: (1.00), 3857: (1.00), 3858: (1.00), 3859: (1.00), 3860: (1.00), 3861: (1.00), 3862: (1.00), 3863: (1.00), 3864: (1.00), 3865: (1.00), 3866: (1.00), 3867: (1.00), 3868: (1.07), 3869: (1.09), 3870: (1.01), 3871: (1.00), 3872: (1.00), 3873: (1.00), 3874: (1.00), 3875: (1.00), 3876: (1.00), 3877: (1.00), 3878: (1.00), 3879: (1.00), 3880: (1.00), 3881: (1.00), 3882: (1.00), 3883: (1.00), 3884: (1.00), 3885: (1.00), 3886: (1.00), 3887: (1.00), 3888: (1.00), 3889: (1.00), 3890: (1.00), 3891: (1.00), 3892: (1.00), 3893: (1.00), 3894: (1.00), 3895: (1.00), 3896: (1.09), 3897: (1.04), 3898: (1.00), 3899: (1.00), 3900: (1.00), 3901: (1.00), 3902: (1.00), 3903: (1.00), 3904: (1.00), 3905: (1.00), 3906: (1.00), 3907: (1.00), 3908: (1.00), 3909: (1.00), 3910: (1.00), 3911: (1.00), 3912: (1.00), 3913: (1.00), 3914: (1.00), 3915: (1.00), 3916: (1.00), 3917: (1.00), 3918: (1.00), 3919: (1.00), 3920: (1.00), 3921: (1.00), 3922: (1.00), 3923: (1.02), 3924: (1.10), 3925: (1.10), 3926: (1.00), 3927: (1.00), 3928: (1.00), 3929: (1.00), 3930: (1.00), 3931: (1.00), 3932: (1.00), 3933: (1.00), 3934: (1.00), 3935: (1.00), 3936: (1.00), 3937: (1.00), 3938: (1.22), 3939: (1.12), 3940: (1.02), 3941: (1.00), 3942: (1.00), 3943: (1.00), 3944: (1.00), 3945: (1.00), 3946: (1.00), 3947: (1.00), 3948: (1.00), 3949: (1.00), 3950: (1.00), 3951: (1.00), 3952: (1.26), 3953: (1.00), 3954: (1.00), 3955: (1.00), 3956: (1.00), 3957: (1.00), 3958: (1.00), 3959: (1.00), 3960: (1.00), 3961: (1.00), 3962: (1.00), 3963: (1.00), 3964: (1.00), 3965: (1.00), 3966: (1.00), 3967: (1.00), 3968: (1.00), 3969: (1.00), 3970: (1.00), 3971: (1.00), 3972: (1.00), 3973: (1.01), 3974: (1.16), 3975: (1.06), 3976: (1.01), 3977: (1.00), 3978: (1.00), 3979: (1.15), 3980: (1.11), 3981: (1.04), 3982: (1.00), 3983: (1.00), 3984: (1.00), 3985: (1.00), 3986: (1.00), 3987: (1.00), 3988: (1.00), 3989: (1.00), 3990: (1.00), 3991: (1.00), 3992: (1.00), 3993: (1.13), 3994: (1.03), 3995: (1.04), 3996: (1.00), 3997: (1.00), 3998: (1.00), 3999: (1.00), 4000: (1.00), 4001: (1.00), 4002: (1.00), 4003: (1.00), 4004: (1.00), 4005: (1.00), 4006: (1.01), 4007: (1.14), 4008: (1.00), 4009: (1.00), 4010: (1.00), 4011: (1.00), 4012: (1.00), 4013: (1.00), 4014: (1.00), 4015: (1.00), 4016: (1.00), 4017: (1.00), 4018: (1.00), 4019: (1.00), 4020: (1.00), 4021: (1.00), 4022: (1.00), 4023: (1.00), 4024: (1.00), 4025: (1.00), 4026: (1.00), 4027: (1.00), 4028: (1.00), 4029: (1.00), 4030: (1.00), 4031: (1.00), 4032: (1.00), 4033: (1.00), 4034: (1.00), 4035: (1.00), 4036: (1.00), 4037: (1.00), 4038: (1.00), 4039: (1.00), 4040: (1.00), 4041: (1.14), 4042: (1.00), 4043: (1.00), 4044: (1.00), 4045: (1.00), 4046: (1.00), 4047: (1.33), 4048: (1.09), 4049: (1.07), 4050: (1.02), 4051: (1.01), 4052: (1.01), 4053: (1.00), 4054: (1.19), 4055: (1.09), 4056: (1.08), 4057: (1.03), 4058: (1.00), 4059: (1.00), 4060: (1.07), 4061: (1.03), 4062: (1.00), 4063: (1.00), 4064: (1.00), 4065: (1.00), 4066: (1.01), 4067: (1.11), 4068: (1.04), 4069: (1.05), 4070: (1.01), 4071: (1.01), 4072: (1.01), 4073: (1.29), 4074: (1.17), 4075: (1.04), 4076: (1.03), 4077: (1.01), 4078: (1.01), 4079: (1.02), 4080: (1.20), 4081: (1.04), 4082: (1.02), 4083: (1.00), 4084: (1.00), 4085: (1.00), 4086: (1.01), 4087: (1.07), 4088: (1.02), 4089: (1.00), 4090: (1.00), 4091: (1.00), 4092: (1.00), 4093: (1.00), 4094: (1.00), 4095: (1.00), 4096: (1.00), 4097: (1.00), 4098: (1.00), 4099: (1.00), 4100: (1.00), 4101: (1.00), 4102: (1.00), 4103: (1.00), 4104: (1.00), 4105: (1.00), 4106: (1.00), 4107: (1.00), 4108: (1.00), 4109: (1.00), 4110: (1.00), 4111: (1.00), 4112: (1.00), 4113: (1.00), 4114: (1.00), 4115: (1.05), 4116: (1.03), 4117: (1.00), 4118: (1.00), 4119: (1.00), 4120: (1.00), 4121: (1.00), 4122: (1.00), 4123: (1.00), 4124: (1.00), 4125: (1.00), 4126: (1.00), 4127: (1.00), 4128: (1.00), 4129: (1.00), 4130: (1.00), 4131: (1.00), 4132: (1.00), 4133: (1.00), 4134: (1.00), 4135: (1.00), 4136: (1.00), 4137: (1.06), 4138: (1.09), 4139: (1.04), 4140: (1.01), 4141: (1.01), 4142: (1.19), 4143: (1.03), 4144: (1.03), 4145: (1.00), 4146: (1.00), 4147: (1.00), 4148: (1.00), 4149: (1.00), 4150: (1.00), 4151: (1.00), 4152: (1.00), 4153: (1.00), 4154: (1.00), 4155: (1.00), 4156: (1.16), 4157: (1.10), 4158: (1.04), 4159: (1.00), 4160: (1.00), 4161: (1.00), 4162: (1.00), 4163: (1.00), 4164: (1.00), 4165: (1.00), 4166: (1.00), 4167: (1.00), 4168: (1.00), 4169: (1.13), 4170: (1.01), 4171: (1.00), 4172: (1.00), 4173: (1.00), 4174: (1.00), 4175: (1.00), 4176: (1.00), 4177: (1.00), 4178: (1.00), 4179: (1.00), 4180: (1.00), 4181: (1.00), 4182: (1.00), 4183: (1.00), 4184: (1.00), 4185: (1.00), 4186: (1.00), 4187: (1.00), 4188: (1.00), 4189: (1.00), 4190: (1.00), 4191: (1.00), 4192: (1.00), 4193: (1.00), 4194: (1.00), 4195: (1.00), 4196: (1.00), 4197: (1.14), 4198: (1.05), 4199: (1.01), 4200: (1.00), 4201: (1.00), 4202: (1.00), 4203: (1.00), 4204: (1.00), 4205: (1.00), 4206: (1.00), 4207: (1.00), 4208: (1.00), 4209: (1.00), 4210: (1.00), 4211: (1.00), 4212: (1.00), 4213: (1.00), 4214: (1.00), 4215: (1.00), 4216: (1.00), 4217: (1.00), 4218: (1.00), 4219: (1.00), 4220: (1.00), 4221: (1.00), 4222: (1.00), 4223: (1.00), 4224: (1.00), 4225: (1.20), 4226: (1.00), 4227: (1.00), 4228: (1.00), 4229: (1.00), 4230: (1.00), 4231: (1.00), 4232: (1.00), 4233: (1.00), 4234: (1.00), 4235: (1.00), 4236: (1.00), 4237: (1.00), 4238: (1.00), 4239: (1.00), 4240: (1.00), 4241: (1.00), 4242: (1.00), 4243: (1.00), 4244: (1.00), 4245: (1.00), 4246: (1.00), 4247: (1.00), 4248: (1.00), 4249: (1.00), 4250: (1.00), 4251: (1.00), 4252: (1.00), 4253: (1.12), 4254: (1.07), 4255: (1.02), 4256: (1.00), 4257: (1.00), 4258: (1.00), 4259: (1.00), 4260: (1.00), 4261: (1.00), 4262: (1.00), 4263: (1.00), 4264: (1.00), 4265: (1.00), 4266: (1.08), 4267: (1.16), 4268: (1.08), 4269: (1.00), 4270: (1.00), 4271: (1.00), 4272: (1.00), 4273: (1.00), 4274: (1.00), 4275: (1.00), 4276: (1.00), 4277: (1.00), 4278: (1.00), 4279: (1.00), 4280: (1.14), 4281: (1.02), 4282: (1.00), 4283: (1.00), 4284: (1.00), 4285: (1.00), 4286: (1.00), 4287: (1.00), 4288: (1.00), 4289: (1.00), 4290: (1.00), 4291: (1.00), 4292: (1.00), 4293: (1.00), 4294: (1.00), 4295: (1.00), 4296: (1.00), 4297: (1.00), 4298: (1.00), 4299: (1.00), 4300: (1.00), 4301: (1.00), 4302: (1.00), 4303: (1.00), 4304: (1.00), 4305: (1.00), 4306: (1.00), 4307: (1.00), 4308: (1.01), 4309: (1.12), 4310: (1.06), 4311: (1.00), 4312: (1.00), 4313: (1.00), 4314: (1.00), 4315: (1.00), 4316: (1.00), 4317: (1.00), 4318: (1.00), 4319: (1.00), 4320: (1.00), 4321: (1.00), 4322: (1.00), 4323: (1.00), 4324: (1.00), 4325: (1.00), 4326: (1.00), 4327: (1.00), 4328: (1.00), 4329: (1.00), 4330: (1.00), 4331: (1.00), 4332: (1.00), 4333: (1.00), 4334: (1.00), 4335: (1.00), 4336: (1.00), 4337: (1.12), 4338: (1.01), 4339: (1.00), 4340: (1.00), 4341: (1.00), 4342: (1.00), 4343: (1.00), 4344: (1.00), 4345: (1.00), 4346: (1.00), 4347: (1.00), 4348: (1.00), 4349: (1.00), 4350: (1.00), 4351: (1.00), 4352: (1.00), 4353: (1.00), 4354: (1.00), 4355: (1.00), 4356: (1.00), 4357: (1.00), 4358: (1.00), 4359: (1.00), 4360: (1.00), 4361: (1.00), 4362: (1.00), 4363: (1.00), 4364: (1.00), 4365: (1.12), 4366: (1.05), 4367: (1.03), 4368: (1.00), 4369: (1.00), 4370: (1.00), 4371: (1.00), 4372: (1.00), 4373: (1.00), 4374: (1.00), 4375: (1.00), 4376: (1.00), 4377: (1.00), 4378: (1.00), 4379: (1.15), 4380: (1.08), 4381: (1.03), 4382: (1.00), 4383: (1.00), 4384: (1.00), 4385: (1.00), 4386: (1.00), 4387: (1.00), 4388: (1.00), 4389: (1.00), 4390: (1.00), 4391: (1.00), 4392: (1.00), 4393: (1.21), 4394: (1.00), 4395: (1.00), 4396: (1.00), 4397: (1.00), 4398: (1.00), 4399: (1.00), 4400: (1.00), 4401: (1.00), 4402: (1.00), 4403: (1.00), 4404: (1.00), 4405: (1.00), 4406: (1.00), 4407: (1.00), 4408: (1.00), 4409: (1.00), 4410: (1.00), 4411: (1.00), 4412: (1.00), 4413: (1.00), 4414: (1.00), 4415: (1.00), 4416: (1.00), 4417: (1.00), 4418: (1.00), 4419: (1.00), 4420: (1.11), 4421: (1.12), 4422: (1.01), 4423: (1.00), 4424: (1.00), 4425: (1.00), 4426: (1.00), 4427: (1.00), 4428: (1.00), 4429: (1.00), 4430: (1.00), 4431: (1.00), 4432: (1.00), 4433: (1.00), 4434: (1.00), 4435: (1.00), 4436: (1.00), 4437: (1.00), 4438: (1.00), 4439: (1.00), 4440: (1.00), 4441: (1.00), 4442: (1.00), 4443: (1.00), 4444: (1.00), 4445: (1.00), 4446: (1.00), 4447: (1.00), 4448: (1.00), 4449: (1.15), 4450: (1.00), 4451: (1.00), 4452: (1.00), 4453: (1.00), 4454: (1.00), 4455: (1.00), 4456: (1.00), 4457: (1.00), 4458: (1.00), 4459: (1.00), 4460: (1.00), 4461: (1.00), 4462: (1.00), 4463: (1.00), 4464: (1.00), 4465: (1.00), 4466: (1.00), 4467: (1.00), 4468: (1.00), 4469: (1.00), 4470: (1.00), 4471: (1.00), 4472: (1.00), 4473: (1.00), 4474: (1.00), 4475: (1.00), 4476: (1.00), 4477: (1.00), 4478: (1.00), 4479: (1.00), 4480: (1.00), 4481: (1.00), 4482: (1.00), 4483: (1.10), 4484: (1.01), 4485: (1.00), 4486: (1.00), 4487: (1.00), 4488: (1.00), 4489: (1.02), 4490: (1.03), 4491: (1.10), 4492: (1.01), 4493: (1.02), 4494: (1.00), 4495: (1.00), 4496: (1.00), 4497: (1.02), 4498: (1.11), 4499: (1.02), 4500: (1.01), 4501: (1.00), 4502: (1.00), 4503: (1.00), 4504: (1.00), 4505: (1.00), 4506: (1.00), 4507: (1.00), 4508: (1.00), 4509: (1.00), 4510: (1.06), 4511: (1.02), 4512: (1.00), 4513: (1.00), 4514: (1.00), 4515: (1.00), 4516: (1.18), 4517: (1.00), 4518: (1.00), 4519: (1.00), 4520: (1.00), 4521: (1.00), 4522: (1.00), 4523: (1.00), 4524: (1.00), 4525: (1.00), 4526: (1.00), 4527: (1.00), 4528: (1.00), 4529: (1.00), 4530: (1.10), 4531: (1.06), 4532: (1.05), 4533: (1.00), 4534: (1.00), 4535: (1.00), 4536: (1.00), 4537: (1.00), 4538: (1.00), 4539: (1.00), 4540: (1.00), 4541: (1.00), 4542: (1.00), 4543: (1.00), 4544: (1.00), 4545: (1.00), 4546: (1.00), 4547: (1.00), 4548: (1.00), 4549: (1.00), 4550: (1.00), 4551: (1.00), 4552: (1.00), 4553: (1.00), 4554: (1.00), 4555: (1.00), 4556: (1.00), 4557: (1.00), 4558: (1.00), 4559: (1.10), 4560: (1.00), 4561: (1.00), 4562: (1.00), 4563: (1.00), 4564: (1.00), 4565: (1.00), 4566: (1.00), 4567: (1.00), 4568: (1.00), 4569: (1.00), 4570: (1.00), 4571: (1.00), 4572: (1.00), 4573: (1.00), 4574: (1.00), 4575: (1.00), 4576: (1.00), 4577: (1.00), 4578: (1.00), 4579: (1.00), 4580: (1.00), 4581: (1.02), 4582: (1.10), 4583: (1.03), 4584: (1.02), 4585: (1.00), 4586: (1.32), 4587: (1.03), 4588: (1.04), 4589: (1.00), 4590: (1.00), 4591: (1.00), 4592: (1.00), 4593: (1.00), 4594: (1.00), 4595: (1.00), 4596: (1.00), 4597: (1.00), 4598: (1.00), 4599: (1.00), 4600: (1.06), 4601: (1.09), 4602: (1.06), 4603: (1.00), 4604: (1.00), 4605: (1.00), 4606: (1.00), 4607: (1.00), 4608: (1.00), 4609: (1.00), 4610: (1.00), 4611: (1.00), 4612: (1.00), 4613: (1.00), 4614: (1.00), 4615: (1.00), 4616: (1.00), 4617: (1.00), 4618: (1.00), 4619: (1.00), 4620: (1.00), 4621: (1.14), 4622: (1.00), 4623: (1.00), 4624: (1.00), 4625: (1.00), 4626: (1.00), 4627: (1.00), 4628: (1.04), 4629: (1.00), 4630: (1.00), 4631: (1.00), 4632: (1.00), 4633: (1.00), 4634: (1.00), 4635: (1.00), 4636: (1.00), 4637: (1.00), 4638: (1.00), 4639: (1.00), 4640: (1.00), 4641: (1.00), 4642: (1.28), 4643: (1.05), 4644: (1.01), 4645: (1.00), 4646: (1.00), 4647: (1.00), 4648: (1.00), 4649: (1.00), 4650: (1.00), 4651: (1.00), 4652: (1.00), 4653: (1.00), 4654: (1.00), 4655: (1.00), 4656: (1.00), 4657: (1.00), 4658: (1.00), 4659: (1.00), 4660: (1.00), 4661: (1.00), 4662: (1.00), 4663: (1.00), 4664: (1.00), 4665: (1.00), 4666: (1.00), 4667: (1.00), 4668: (1.00), 4669: (1.00), 4670: (1.06), 4671: (1.00), 4672: (1.00), 4673: (1.00), 4674: (1.00), 4675: (1.00), 4676: (1.00), 4677: (1.00), 4678: (1.00), 4679: (1.00), 4680: (1.00), 4681: (1.00), 4682: (1.00), 4683: (1.00), 4684: (1.00), 4685: (1.00), 4686: (1.00), 4687: (1.00), 4688: (1.00), 4689: (1.00), 4690: (1.00), 4691: (1.00), 4692: (1.00), 4693: (1.18), 4694: (1.04), 4695: (1.02), 4696: (1.00), 4697: (1.17), 4698: (1.06), 4699: (1.09), 4700: (1.00), 4701: (1.00), 4702: (1.00), 4703: (1.00), 4704: (1.00), 4705: (1.00), 4706: (1.00), 4707: (1.00), 4708: (1.00), 4709: (1.00), 4710: (1.00), 4711: (1.18), 4712: (1.05), 4713: (1.05), 4714: (1.00), 4715: (1.00), 4716: (1.00), 4717: (1.00), 4718: (1.00), 4719: (1.00), 4720: (1.00), 4721: (1.00), 4722: (1.00), 4723: (1.00), 4724: (1.05), 4725: (1.00), 4726: (1.00), 4727: (1.00), 4728: (1.00), 4729: (1.00), 4730: (1.00), 4731: (1.00), 4732: (1.00), 4733: (1.22), 4734: (1.03), 4735: (1.02), 4736: (1.00), 4737: (1.00), 4738: (1.00), 4739: (1.00), 4740: (1.00), 4741: (1.00), 4742: (1.00), 4743: (1.00), 4744: (1.00), 4745: (1.00), 4746: (1.00), 4747: (1.00), 4748: (1.00), 4749: (1.00), 4750: (1.00), 4751: (1.11), 4752: (1.03), 4753: (1.03), 4754: (1.00), 4755: (1.01), 4756: (1.00), 4757: (1.00), 4758: (1.00), 4759: (1.00), 4760: (1.00), 4761: (1.00), 4762: (1.00), 4763: (1.00), 4764: (1.00), 4765: (1.00), 4766: (1.00), 4767: (1.00), 4768: (1.00), 4769: (1.00), 4770: (1.00), 4771: (1.00), 4772: (1.00), 4773: (1.00), 4774: (1.00), 4775: (1.00), 4776: (1.00), 4777: (1.00), 4778: (1.00), 4779: (1.00), 4780: (1.00), 4781: (1.09), 4782: (1.00), 4783: (1.00), 4784: (1.00), 4785: (1.00), 4786: (1.00), 4787: (1.00), 4788: (1.00), 4789: (1.00), 4790: (1.00), 4791: (1.00), 4792: (1.00), 4793: (1.00), 4794: (1.00), 4795: (1.00), 4796: (1.00), 4797: (1.00), 4798: (1.00), 4799: (1.00), 4800: (1.00), 4801: (1.00), 4802: (1.10), 4803: (1.26), 4804: (1.06), 4805: (1.06), 4806: (1.06), 4807: (1.00), 4808: (1.01), 4809: (1.22), 4810: (1.04), 4811: (1.01), 4812: (1.00), 4813: (1.00), 4814: (1.00), 4815: (1.00), 4816: (1.00), 4817: (1.00), 4818: (1.00), 4819: (1.00), 4820: (1.00), 4821: (1.00), 4822: (1.00), 4823: (1.00), 4824: (1.00), 4825: (1.00), 4826: (1.00), 4827: (1.00), 4828: (1.00), 4829: (1.00), 4830: (1.00), 4831: (1.00), 4832: (1.00), 4833: (1.00), 4834: (1.00), 4835: (1.00), 4836: (1.00), 4837: (1.01), 4838: (1.00), 4839: (1.00), 4840: (1.00), 4841: (1.00), 4842: (1.00), 4843: (1.00), 4844: (1.00), 4845: (1.00), 4846: (1.00), 4847: (1.00), 4848: (1.00), 4849: (1.00), 4850: (1.00), 4851: (1.00), 4852: (1.00), 4853: (1.00), 4854: (1.00), 4855: (1.00), 4856: (1.00), 4857: (1.00), 4858: (1.00), 4859: (1.00), 4860: (1.00), 4861: (1.00), 4862: (1.00), 4863: (1.00), 4864: (1.00), 4865: (1.02), 4866: (1.11), 4867: (1.07), 4868: (1.00), 4869: (1.00), 4870: (1.00), 4871: (1.00), 4872: (1.00), 4873: (1.00), 4874: (1.00), 4875: (1.00), 4876: (1.00), 4877: (1.00), 4878: (1.00), 4879: (1.00), 4880: (1.00), 4881: (1.00), 4882: (1.00), 4883: (1.00), 4884: (1.00), 4885: (1.00), 4886: (1.00), 4887: (1.00), 4888: (1.00), 4889: (1.00), 4890: (1.00), 4891: (1.00), 4892: (1.00), 4893: (1.00), 4894: (1.03), 4895: (1.00), 4896: (1.00), 4897: (1.00), 4898: (1.00), 4899: (1.00), 4900: (1.00), 4901: (1.00), 4902: (1.00), 4903: (1.00), 4904: (1.00), 4905: (1.00), 4906: (1.00), 4907: (1.00), 4908: (1.00), 4909: (1.00), 4910: (1.00), 4911: (1.00), 4912: (1.00), 4913: (1.00), 4914: (1.00), 4915: (1.00), 4916: (1.00), 4917: (1.00), 4918: (1.00), 4919: (1.00), 4920: (1.00), 4921: (1.00), 4922: (1.09), 4923: (1.07), 4924: (1.09), 4925: (1.00), 4926: (1.00), 4927: (1.00), 4928: (1.00), 4929: (1.00), 4930: (1.00), 4931: (1.00), 4932: (1.00), 4933: (1.00), 4934: (1.00), 4935: (1.00), 4936: (1.00), 4937: (1.00), 4938: (1.00), 4939: (1.00), 4940: (1.00), 4941: (1.00), 4942: (1.00), 4943: (1.00), 4944: (1.00), 4945: (1.00), 4946: (1.00), 4947: (1.00), 4948: (1.00), 4949: (1.00), 4950: (1.06), 4951: (1.00), 4952: (1.00), 4953: (1.00), 4954: (1.00), 4955: (1.00), 4956: (1.00), 4957: (1.00), 4958: (1.00), 4959: (1.00), 4960: (1.00), 4961: (1.00), 4962: (1.00), 4963: (1.00), 4964: (1.03), 4965: (1.02), 4966: (1.00), 4967: (1.00), 4968: (1.00), 4969: (1.00), 4970: (1.00), 4971: (1.00), 4972: (1.00), 4973: (1.00), 4974: (1.00), 4975: (1.00), 4976: (1.00), 4977: (1.00), 4978: (1.00), 4979: (1.00)"
    translation_x = "0:(0)"#@param {type:"string"}
    translation_y = "0:(0)"#@param {type:"string"}
    translation_z = "0:(10)"#@param {type:"string"}
    rotation_3d_x = "0:(0)"#@param {type:"string"}
    rotation_3d_y = "0:(0)"#@param {type:"string"}
    rotation_3d_z = "0:(0)"#@param {type:"string"}
    flip_2d_perspective = False #@param {type:"boolean"}
    perspective_flip_theta = "0:(0)"#@param {type:"string"}
    perspective_flip_phi = "0:(t%15)"#@param {type:"string"}
    perspective_flip_gamma = "0:(0)"#@param {type:"string"}
    perspective_flip_fv = "0:(53)"#@param {type:"string"}
    noise_schedule = "0: (0.02)"#@param {type:"string"}
    strength_schedule = "0: (0.65)"#@param {type:"string"}
    contrast_schedule = "0: (1.0)"#@param {type:"string"}

    #@markdown ####**Coherence:**
    color_coherence = 'Match Frame 0 LAB' #@param ['None', 'Match Frame 0 HSV', 'Match Frame 0 LAB', 'Match Frame 0 RGB'] {type:'string'}
    diffusion_cadence = '2' #@param ['1','2','3','4','5','6','7','8'] {type:'string'}

    #@markdown ####**3D Depth Warping:**
    use_depth_warping = True #@param {type:"boolean"}
    midas_weight = 0.3#@param {type:"number"}
    near_plane = 200
    far_plane = 10000
    fov = 40#@param {type:"number"}
    padding_mode = 'border'#@param ['border', 'reflection', 'zeros'] {type:'string'}
    sampling_mode = 'bicubic'#@param ['bicubic', 'bilinear', 'nearest'] {type:'string'}
    save_depth_maps = False #@param {type:"boolean"}

    #@markdown ####**Video Input:**
    video_init_path ='/content/video_in.mp4'#@param {type:"string"}
    extract_nth_frame = 1#@param {type:"number"}
    overwrite_extracted_frames = True #@param {type:"boolean"}
    use_mask_video = False #@param {type:"boolean"}
    video_mask_path ='/content/video_in.mp4'#@param {type:"string"}

    #@markdown ####**Interpolation:**
    interpolate_key_frames = False #@param {type:"boolean"}
    interpolate_x_frames = 4 #@param {type:"number"}
    
    #@markdown ####**Resume Animation:**
    resume_from_timestring = False #@param {type:"boolean"}
    resume_timestring = "20220829210106" #@param {type:"string"}

    return locals()

class DeformAnimKeys():
    def __init__(self, anim_args):
        self.angle_series = get_inbetweens(parse_key_frames(anim_args.angle), anim_args.max_frames)
        self.zoom_series = get_inbetweens(parse_key_frames(anim_args.zoom), anim_args.max_frames)
        self.translation_x_series = get_inbetweens(parse_key_frames(anim_args.translation_x), anim_args.max_frames)
        self.translation_y_series = get_inbetweens(parse_key_frames(anim_args.translation_y), anim_args.max_frames)
        self.translation_z_series = get_inbetweens(parse_key_frames(anim_args.translation_z), anim_args.max_frames)
        self.rotation_3d_x_series = get_inbetweens(parse_key_frames(anim_args.rotation_3d_x), anim_args.max_frames)
        self.rotation_3d_y_series = get_inbetweens(parse_key_frames(anim_args.rotation_3d_y), anim_args.max_frames)
        self.rotation_3d_z_series = get_inbetweens(parse_key_frames(anim_args.rotation_3d_z), anim_args.max_frames)
        self.perspective_flip_theta_series = get_inbetweens(parse_key_frames(anim_args.perspective_flip_theta), anim_args.max_frames)
        self.perspective_flip_phi_series = get_inbetweens(parse_key_frames(anim_args.perspective_flip_phi), anim_args.max_frames)
        self.perspective_flip_gamma_series = get_inbetweens(parse_key_frames(anim_args.perspective_flip_gamma), anim_args.max_frames)
        self.perspective_flip_fv_series = get_inbetweens(parse_key_frames(anim_args.perspective_flip_fv), anim_args.max_frames)
        self.noise_schedule_series = get_inbetweens(parse_key_frames(anim_args.noise_schedule), anim_args.max_frames)
        self.strength_schedule_series = get_inbetweens(parse_key_frames(anim_args.strength_schedule), anim_args.max_frames)
        self.contrast_schedule_series = get_inbetweens(parse_key_frames(anim_args.contrast_schedule), anim_args.max_frames)


def get_inbetweens(key_frames, max_frames, integer=False, interp_method='Linear'):
    import numexpr
    key_frame_series = pd.Series([np.nan for a in range(max_frames)])
    
    for i in range(0, max_frames):
        if i in key_frames:
            value = key_frames[i]
            value_is_number = check_is_number(value)
            # if it's only a number, leave the rest for the default interpolation
            if value_is_number:
                t = i
                key_frame_series[i] = value
        if not value_is_number:
            t = i
            key_frame_series[i] = numexpr.evaluate(value)
    key_frame_series = key_frame_series.astype(float)
    
    if interp_method == 'Cubic' and len(key_frames.items()) <= 3:
      interp_method = 'Quadratic'    
    if interp_method == 'Quadratic' and len(key_frames.items()) <= 2:
      interp_method = 'Linear'
          
    key_frame_series[0] = key_frame_series[key_frame_series.first_valid_index()]
    key_frame_series[max_frames-1] = key_frame_series[key_frame_series.last_valid_index()]
    key_frame_series = key_frame_series.interpolate(method=interp_method.lower(), limit_direction='both')
    if integer:
        return key_frame_series.astype(int)
    return key_frame_series

def parse_key_frames(string, prompt_parser=None):
    # because math functions (i.e. sin(t)) can utilize brackets 
    # it extracts the value in form of some stuff
    # which has previously been enclosed with brackets and
    # with a comma or end of line existing after the closing one
    pattern = r'((?P<frame>[0-9]+):[\s]*\((?P<param>[\S\s]*?)\)([,][\s]?|[\s]?$))'
    frames = dict()
    for match_object in re.finditer(pattern, string):
        frame = int(match_object.groupdict()['frame'])
        param = match_object.groupdict()['param']
        if prompt_parser:
            frames[frame] = prompt_parser(param)
        else:
            frames[frame] = param
    if frames == {} and len(string) != 0:
        raise RuntimeError('Key Frame string not correctly formatted')
    return frames

### Prompts
`animation_mode: None` batches on list of *prompts*. `animation_mode: 2D` uses *animation_prompts* key frame sequence

In [8]:

prompts = [
    "a beautiful forest by Asher Brown Durand, trending on Artstation", # the first prompt I want
    "a beautiful portrait of a woman by Artgerm, trending on Artstation", # the second prompt I want
    #"this prompt I don't want it I commented it out",
    #"a nousr robot, trending on Artstation", # use "nousr robot" with the robot diffusion model (see model_checkpoint setting)
    #"touhou 1girl komeiji_koishi portrait, green hair", # waifu diffusion prompts can use danbooru tag groups (see model_checkpoint)
    #"this prompt has weights if prompt weighting enabled:2 can also do negative:-2", # (see prompt_weighting)
]

animation_prompts = {
    0: "dark side of the moon album art, pink floyd, prismatic, psychadelic, trippy, 1970s, trending on artstation"
    
}


# Run

In [9]:
#@markdown **Load Settings**
override_settings_with_file = False #@param {type:"boolean"}
custom_settings_file = "/content/drive/MyDrive/Settings.txt"#@param {type:"string"}

def DeforumArgs():
    #@markdown **Image Settings**
    W = 640 #@param
    H = 320 #@param
    W, H = map(lambda x: x - x % 64, (W, H))  # resize to integer multiple of 64

    #@markdown **Sampling Settings**
    seed = -1 #@param
    sampler = 'klms' #@param ["klms","dpm2","dpm2_ancestral","heun","euler","euler_ancestral","plms", "ddim"]
    steps = 50 #@param
    scale = 7 #@param
    ddim_eta = 0.0 #@param
    dynamic_threshold = None
    static_threshold = None   

    #@markdown **Save & Display Settings**
    save_samples = True #@param {type:"boolean"}
    save_settings = True #@param {type:"boolean"}
    display_samples = True #@param {type:"boolean"}
    save_sample_per_step = False #@param {type:"boolean"}
    show_sample_per_step = False #@param {type:"boolean"}

    #@markdown **Prompt Settings**
    prompt_weighting = False #@param {type:"boolean"}
    normalize_prompt_weights = True #@param {type:"boolean"}
    log_weighted_subprompts = False #@param {type:"boolean"}

    #@markdown **Batch Settings**
    n_batch = 1 #@param
    batch_name = "StableFun" #@param {type:"string"}
    filename_format = "{timestring}_{index}_{prompt}.png" #@param ["{timestring}_{index}_{seed}.png","{timestring}_{index}_{prompt}.png"]
    seed_behavior = "iter" #@param ["iter","fixed","random"]
    make_grid = False #@param {type:"boolean"}
    grid_rows = 2 #@param 
    outdir = get_output_folder(output_path, batch_name)

    #@markdown **Init Settings**
    use_init = False #@param {type:"boolean"}
    strength = 0.0 #@param {type:"number"}
    strength_0_no_init = True # Set the strength to 0 automatically when no init image is used
    init_image = "https://cdn.pixabay.com/photo/2022/07/30/13/10/green-longhorn-beetle-7353749_1280.jpg" #@param {type:"string"}
    # Whiter areas of the mask are areas that change more
    use_mask = False #@param {type:"boolean"}
    use_alpha_as_mask = False # use the alpha channel of the init image as the mask
    mask_file = "https://www.filterforge.com/wiki/images/archive/b/b7/20080927223728%21Polygonal_gradient_thumb.jpg" #@param {type:"string"}
    invert_mask = False #@param {type:"boolean"}
    # Adjust mask image, 1.0 is no adjustment. Should be positive numbers.
    mask_brightness_adjust = 1.0  #@param {type:"number"}
    mask_contrast_adjust = 1.0  #@param {type:"number"}
    # Overlay the masked image at the end of the generation so it does not get degraded by encoding and decoding
    overlay_mask = True  # {type:"boolean"}
    # Blur edges of final overlay mask, if used. Minimum = 0 (no blur)
    mask_overlay_blur = 5 # {type:"number"}

    n_samples = 1 # doesnt do anything
    precision = 'autocast' 
    C = 4
    f = 8

    prompt = ""
    timestring = ""
    init_latent = None
    init_sample = None
    init_c = None

    return locals()



def next_seed(args):
    if args.seed_behavior == 'iter':
        args.seed += 1
    elif args.seed_behavior == 'fixed':
        pass # always keep seed the same
    else:
        args.seed = random.randint(0, 2**32 - 1)
    return args.seed

def render_image_batch(args):
    args.prompts = {k: f"{v:05d}" for v, k in enumerate(prompts)}
    
    # create output folder for the batch
    os.makedirs(args.outdir, exist_ok=True)
    if args.save_settings or args.save_samples:
        print(f"Saving to {os.path.join(args.outdir, args.timestring)}_*")

    # save settings for the batch
    if args.save_settings:
        filename = os.path.join(args.outdir, f"{args.timestring}_settings.txt")
        with open(filename, "w+", encoding="utf-8") as f:
            json.dump(dict(args.__dict__), f, ensure_ascii=False, indent=4)

    index = 0
    
    # function for init image batching
    init_array = []
    if args.use_init:
        if args.init_image == "":
            raise FileNotFoundError("No path was given for init_image")
        if args.init_image.startswith('http://') or args.init_image.startswith('https://'):
            init_array.append(args.init_image)
        elif not os.path.isfile(args.init_image):
            if args.init_image[-1] != "/": # avoids path error by adding / to end if not there
                args.init_image += "/" 
            for image in sorted(os.listdir(args.init_image)): # iterates dir and appends images to init_array
                if image.split(".")[-1] in ("png", "jpg", "jpeg"):
                    init_array.append(args.init_image + image)
        else:
            init_array.append(args.init_image)
    else:
        init_array = [""]

    # when doing large batches don't flood browser with images
    clear_between_batches = args.n_batch >= 32

    for iprompt, prompt in enumerate(prompts):  
        args.prompt = prompt
        print(f"Prompt {iprompt+1} of {len(prompts)}")
        print(f"{args.prompt}")

        all_images = []

        for batch_index in range(args.n_batch):
            if clear_between_batches and batch_index % 32 == 0: 
                display.clear_output(wait=True)            
            print(f"Batch {batch_index+1} of {args.n_batch}")
            
            for image in init_array: # iterates the init images
                args.init_image = image
                results = generate(args)
                for image in results:
                    if args.make_grid:
                        all_images.append(T.functional.pil_to_tensor(image))
                    if args.save_samples:
                        if args.filename_format == "{timestring}_{index}_{prompt}.png":
                            filename = f"{args.timestring}_{index:05}_{sanitize(prompt)[:160]}.png"
                        else:
                            filename = f"{args.timestring}_{index:05}_{args.seed}.png"
                        image.save(os.path.join(args.outdir, filename))
                    if args.display_samples:
                        display.display(image)
                    index += 1
                args.seed = next_seed(args)

        #print(len(all_images))
        if args.make_grid:
            grid = make_grid(all_images, nrow=int(len(all_images)/args.grid_rows))
            grid = rearrange(grid, 'c h w -> h w c').cpu().numpy()
            filename = f"{args.timestring}_{iprompt:05d}_grid_{args.seed}.png"
            grid_image = Image.fromarray(grid.astype(np.uint8))
            grid_image.save(os.path.join(args.outdir, filename))
            display.clear_output(wait=True)            
            display.display(grid_image)


def render_animation(args, anim_args):
    # animations use key framed prompts
    args.prompts = animation_prompts

    # expand key frame strings to values
    keys = DeformAnimKeys(anim_args)

    # resume animation
    start_frame = 0
    if anim_args.resume_from_timestring:
        for tmp in os.listdir(args.outdir):
            if tmp.split("_")[0] == anim_args.resume_timestring:
                start_frame += 1
        start_frame = start_frame - 1

    # create output folder for the batch
    os.makedirs(args.outdir, exist_ok=True)
    print(f"Saving animation frames to {args.outdir}")

    # save settings for the batch
    settings_filename = os.path.join(args.outdir, f"{args.timestring}_settings.txt")
    with open(settings_filename, "w+", encoding="utf-8") as f:
        s = {**dict(args.__dict__), **dict(anim_args.__dict__)}
        json.dump(s, f, ensure_ascii=False, indent=4)
        
    # resume from timestring
    if anim_args.resume_from_timestring:
        args.timestring = anim_args.resume_timestring

    # expand prompts out to per-frame
    prompt_series = pd.Series([np.nan for a in range(anim_args.max_frames)])
    for i, prompt in animation_prompts.items():
        prompt_series[int(i)] = prompt
    prompt_series = prompt_series.ffill().bfill()

    # check for video inits
    using_vid_init = anim_args.animation_mode == 'Video Input'

    # load depth model for 3D
    predict_depths = (anim_args.animation_mode == '3D' and anim_args.use_depth_warping) or anim_args.save_depth_maps
    if predict_depths:
        depth_model = DepthModel(device)
        depth_model.load_midas(models_path)
        if anim_args.midas_weight < 1.0:
            depth_model.load_adabins()
    else:
        depth_model = None
        anim_args.save_depth_maps = False

    # state for interpolating between diffusion steps
    turbo_steps = 1 if using_vid_init else int(anim_args.diffusion_cadence)
    turbo_prev_image, turbo_prev_frame_idx = None, 0
    turbo_next_image, turbo_next_frame_idx = None, 0

    # resume animation
    prev_sample = None
    color_match_sample = None
    if anim_args.resume_from_timestring:
        last_frame = start_frame-1
        if turbo_steps > 1:
            last_frame -= last_frame%turbo_steps
        path = os.path.join(args.outdir,f"{args.timestring}_{last_frame:05}.png")
        img = cv2.imread(path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        prev_sample = sample_from_cv2(img)
        if anim_args.color_coherence != 'None':
            color_match_sample = img
        if turbo_steps > 1:
            turbo_next_image, turbo_next_frame_idx = sample_to_cv2(prev_sample, type=np.float32), last_frame
            turbo_prev_image, turbo_prev_frame_idx = turbo_next_image, turbo_next_frame_idx
            start_frame = last_frame+turbo_steps

    args.n_samples = 1
    frame_idx = start_frame
    while frame_idx < anim_args.max_frames:
        print(f"Rendering animation frame {frame_idx} of {anim_args.max_frames}")
        noise = keys.noise_schedule_series[frame_idx]
        strength = keys.strength_schedule_series[frame_idx]
        contrast = keys.contrast_schedule_series[frame_idx]
        depth = None
        
        # emit in-between frames
        if turbo_steps > 1:
            tween_frame_start_idx = max(0, frame_idx-turbo_steps)
            for tween_frame_idx in range(tween_frame_start_idx, frame_idx):
                tween = float(tween_frame_idx - tween_frame_start_idx + 1) / float(frame_idx - tween_frame_start_idx)
                print(f"  creating in between frame {tween_frame_idx} tween:{tween:0.2f}")

                advance_prev = turbo_prev_image is not None and tween_frame_idx > turbo_prev_frame_idx
                advance_next = tween_frame_idx > turbo_next_frame_idx

                if depth_model is not None:
                    assert(turbo_next_image is not None)
                    depth = depth_model.predict(turbo_next_image, anim_args)

                if anim_args.animation_mode == '2D':
                    if advance_prev:
                        turbo_prev_image = anim_frame_warp_2d(turbo_prev_image, args, anim_args, keys, tween_frame_idx)
                    if advance_next:
                        turbo_next_image = anim_frame_warp_2d(turbo_next_image, args, anim_args, keys, tween_frame_idx)
                else: # '3D'
                    if advance_prev:
                        turbo_prev_image = anim_frame_warp_3d(turbo_prev_image, depth, anim_args, keys, tween_frame_idx)
                    if advance_next:
                        turbo_next_image = anim_frame_warp_3d(turbo_next_image, depth, anim_args, keys, tween_frame_idx)
                turbo_prev_frame_idx = turbo_next_frame_idx = tween_frame_idx

                if turbo_prev_image is not None and tween < 1.0:
                    img = turbo_prev_image*(1.0-tween) + turbo_next_image*tween
                else:
                    img = turbo_next_image

                filename = f"{args.timestring}_{tween_frame_idx:05}.png"
                cv2.imwrite(os.path.join(args.outdir, filename), cv2.cvtColor(img.astype(np.uint8), cv2.COLOR_RGB2BGR))
                if anim_args.save_depth_maps:
                    depth_model.save(os.path.join(args.outdir, f"{args.timestring}_depth_{tween_frame_idx:05}.png"), depth)
            if turbo_next_image is not None:
                prev_sample = sample_from_cv2(turbo_next_image)

        # apply transforms to previous frame
        if prev_sample is not None:
            if anim_args.animation_mode == '2D':
                prev_img = anim_frame_warp_2d(sample_to_cv2(prev_sample), args, anim_args, keys, frame_idx)
            else: # '3D'
                prev_img_cv2 = sample_to_cv2(prev_sample)
                depth = depth_model.predict(prev_img_cv2, anim_args) if depth_model else None
                prev_img = anim_frame_warp_3d(prev_img_cv2, depth, anim_args, keys, frame_idx)

            # apply color matching
            if anim_args.color_coherence != 'None':
                if color_match_sample is None:
                    color_match_sample = prev_img.copy()
                else:
                    prev_img = maintain_colors(prev_img, color_match_sample, anim_args.color_coherence)

            # apply scaling
            contrast_sample = prev_img * contrast
            # apply frame noising
            noised_sample = add_noise(sample_from_cv2(contrast_sample), noise)

            # use transformed previous frame as init for current
            args.use_init = True
            if half_precision:
                args.init_sample = noised_sample.half().to(device)
            else:
                args.init_sample = noised_sample.to(device)
            args.strength = max(0.0, min(1.0, strength))

        # grab prompt for current frame
        args.prompt = prompt_series[frame_idx]
        print(f"{args.prompt} {args.seed}")
        if not using_vid_init:
            print(f"Angle: {keys.angle_series[frame_idx]} Zoom: {keys.zoom_series[frame_idx]}")
            print(f"Tx: {keys.translation_x_series[frame_idx]} Ty: {keys.translation_y_series[frame_idx]} Tz: {keys.translation_z_series[frame_idx]}")
            print(f"Rx: {keys.rotation_3d_x_series[frame_idx]} Ry: {keys.rotation_3d_y_series[frame_idx]} Rz: {keys.rotation_3d_z_series[frame_idx]}")

        # grab init image for current frame
        if using_vid_init:
            init_frame = os.path.join(args.outdir, 'inputframes', f"{frame_idx+1:05}.jpg")            
            print(f"Using video init frame {init_frame}")
            args.init_image = init_frame
            if anim_args.use_mask_video:
                mask_frame = os.path.join(args.outdir, 'maskframes', f"{frame_idx+1:05}.jpg")
                args.mask_file = mask_frame

        # sample the diffusion model
        sample, image = generate(args, frame_idx, return_latent=False, return_sample=True)
        if not using_vid_init:
            prev_sample = sample

        if turbo_steps > 1:
            turbo_prev_image, turbo_prev_frame_idx = turbo_next_image, turbo_next_frame_idx
            turbo_next_image, turbo_next_frame_idx = sample_to_cv2(sample, type=np.float32), frame_idx
            frame_idx += turbo_steps
        else:    
            filename = f"{args.timestring}_{frame_idx:05}.png"
            image.save(os.path.join(args.outdir, filename))
            if anim_args.save_depth_maps:
                if depth is None:
                    depth = depth_model.predict(sample_to_cv2(sample), anim_args)
                depth_model.save(os.path.join(args.outdir, f"{args.timestring}_depth_{frame_idx:05}.png"), depth)
            frame_idx += 1

        display.clear_output(wait=True)
        display.display(image)

        args.seed = next_seed(args)

def vid2frames(video_path, frames_path, n=1, overwrite=True):      
    if not os.path.exists(frames_path) or overwrite: 
      try:
          for f in pathlib.Path(video_in_frame_path).glob('*.jpg'):
              f.unlink()
      except:
          pass
      assert os.path.exists(video_path), f"Video input {video_path} does not exist"
          
      vidcap = cv2.VideoCapture(video_path)
      success,image = vidcap.read()
      count = 0
      t=1
      success = True
      while success:
        if count % n == 0:
            cv2.imwrite(frames_path + os.path.sep + f"{t:05}.jpg" , image)     # save frame as JPEG file
            t += 1
        success,image = vidcap.read()
        count += 1
      print("Converted %d frames" % count)
    else: print("Frames already unpacked")

def render_input_video(args, anim_args):
    # create a folder for the video input frames to live in
    video_in_frame_path = os.path.join(args.outdir, 'inputframes') 
    os.makedirs(video_in_frame_path, exist_ok=True)
    
    # save the video frames from input video
    print(f"Exporting Video Frames (1 every {anim_args.extract_nth_frame}) frames to {video_in_frame_path}...")
    vid2frames(anim_args.video_init_path, video_in_frame_path, anim_args.extract_nth_frame, anim_args.overwrite_extracted_frames)

    # determine max frames from length of input frames
    anim_args.max_frames = len([f for f in pathlib.Path(video_in_frame_path).glob('*.jpg')])
    args.use_init = True
    print(f"Loading {anim_args.max_frames} input frames from {video_in_frame_path} and saving video frames to {args.outdir}")

    if anim_args.use_mask_video:
        # create a folder for the mask video input frames to live in
        mask_in_frame_path = os.path.join(args.outdir, 'maskframes') 
        os.makedirs(mask_in_frame_path, exist_ok=True)

        # save the video frames from mask video
        print(f"Exporting Video Frames (1 every {anim_args.extract_nth_frame}) frames to {mask_in_frame_path}...")
        vid2frames(anim_args.video_mask_path, mask_in_frame_path, anim_args.extract_nth_frame, anim_args.overwrite_extracted_frames)
        args.use_mask = True
        args.overlay_mask = True

    render_animation(args, anim_args)

def render_interpolation(args, anim_args):
    # animations use key framed prompts
    args.prompts = animation_prompts

    # create output folder for the batch
    os.makedirs(args.outdir, exist_ok=True)
    print(f"Saving animation frames to {args.outdir}")

    # save settings for the batch
    settings_filename = os.path.join(args.outdir, f"{args.timestring}_settings.txt")
    with open(settings_filename, "w+", encoding="utf-8") as f:
        s = {**dict(args.__dict__), **dict(anim_args.__dict__)}
        json.dump(s, f, ensure_ascii=False, indent=4)
    
    # Interpolation Settings
    args.n_samples = 1
    args.seed_behavior = 'fixed' # force fix seed at the moment bc only 1 seed is available
    prompts_c_s = [] # cache all the text embeddings

    print(f"Preparing for interpolation of the following...")

    for i, prompt in animation_prompts.items():
      args.prompt = prompt

      # sample the diffusion model
      results = generate(args, return_c=True)
      c, image = results[0], results[1]
      prompts_c_s.append(c) 
      
      # display.clear_output(wait=True)
      display.display(image)
      
      args.seed = next_seed(args)

    display.clear_output(wait=True)
    print(f"Interpolation start...")

    frame_idx = 0

    if anim_args.interpolate_key_frames:
      for i in range(len(prompts_c_s)-1):
        dist_frames = list(animation_prompts.items())[i+1][0] - list(animation_prompts.items())[i][0]
        if dist_frames <= 0:
          print("key frames duplicated or reversed. interpolation skipped.")
          return
        else:
          for j in range(dist_frames):
            # interpolate the text embedding
            prompt1_c = prompts_c_s[i]
            prompt2_c = prompts_c_s[i+1]  
            args.init_c = prompt1_c.add(prompt2_c.sub(prompt1_c).mul(j * 1/dist_frames))

            # sample the diffusion model
            results = generate(args)
            image = results[0]

            filename = f"{args.timestring}_{frame_idx:05}.png"
            image.save(os.path.join(args.outdir, filename))
            frame_idx += 1

            display.clear_output(wait=True)
            display.display(image)

            args.seed = next_seed(args)

    else:
      for i in range(len(prompts_c_s)-1):
        for j in range(anim_args.interpolate_x_frames+1):
          # interpolate the text embedding
          prompt1_c = prompts_c_s[i]
          prompt2_c = prompts_c_s[i+1]  
          args.init_c = prompt1_c.add(prompt2_c.sub(prompt1_c).mul(j * 1/(anim_args.interpolate_x_frames+1)))

          # sample the diffusion model
          results = generate(args)
          image = results[0]

          filename = f"{args.timestring}_{frame_idx:05}.png"
          image.save(os.path.join(args.outdir, filename))
          frame_idx += 1

          display.clear_output(wait=True)
          display.display(image)

          args.seed = next_seed(args)

    # generate the last prompt
    args.init_c = prompts_c_s[-1]
    results = generate(args)
    image = results[0]
    filename = f"{args.timestring}_{frame_idx:05}.png"
    image.save(os.path.join(args.outdir, filename))

    display.clear_output(wait=True)
    display.display(image)
    args.seed = next_seed(args)

    #clear init_c
    args.init_c = None


args_dict = DeforumArgs()
anim_args_dict = DeforumAnimArgs()

if override_settings_with_file:
    print(f"reading custom settings from {custom_settings_file}")
    if not os.path.isfile(custom_settings_file):
        print('The custom settings file does not exist. The in-notebook settings will be used instead')
    else:
        with open(custom_settings_file, "r") as f:
            jdata = json.loads(f.read())
            animation_prompts = jdata["prompts"]
            for i, k in enumerate(args_dict):
                if k in jdata:
                    args_dict[k] = jdata[k]
                else:
                    print(f"key {k} doesn't exist in the custom settings data! using the default value of {args_dict[k]}")
            for i, k in enumerate(anim_args_dict):
                if k in jdata:
                    anim_args_dict[k] = jdata[k]
                else:
                    print(f"key {k} doesn't exist in the custom settings data! using the default value of {anim_args_dict[k]}")
            print(args_dict)
            print(anim_args_dict)

args = SimpleNamespace(**args_dict)
anim_args = SimpleNamespace(**anim_args_dict)

args.timestring = time.strftime('%Y%m%d%H%M%S')
args.strength = max(0.0, min(1.0, args.strength))

if args.seed == -1:
    args.seed = random.randint(0, 2**32 - 1)
if not args.use_init:
    args.init_image = None
if args.sampler == 'plms' and (args.use_init or anim_args.animation_mode != 'None'):
    print(f"Init images aren't supported with PLMS yet, switching to KLMS")
    args.sampler = 'klms'
if args.sampler != 'ddim':
    args.ddim_eta = 0

if anim_args.animation_mode == 'None':
    anim_args.max_frames = 1
elif anim_args.animation_mode == 'Video Input':
    args.use_init = True

# clean up unused memory
gc.collect()
torch.cuda.empty_cache()

# dispatch to appropriate renderer
if anim_args.animation_mode == '2D' or anim_args.animation_mode == '3D':
    render_animation(args, anim_args)
elif anim_args.animation_mode == 'Video Input':
    render_input_video(args, anim_args)
elif anim_args.animation_mode == 'Interpolation':
    render_interpolation(args, anim_args)
else:
    render_image_batch(args)    

Global seed set to 3917189699


Saving animation frames to /home/jnl6052/music/output/2023-05/StableFun
Rendering animation frame 0 of 5000
dark side of the moon album art, pink floyd, prismatic, psychadelic, trippy, 1970s, trending on artstation 3917189699
Angle: 0.0 Zoom: 1.0
Tx: 0.0 Ty: 0.0 Tz: 10.0
Rx: 0.0 Ry: 0.0 Rz: 0.0


  0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
print("done")

# Create video from frames

In [ ]:
# Josh additions
# batch_name = "StableFun" #@param {type:"string"}
# outdir = get_output_folder(output_path, batch_name)
# steps = 50 #@param
# timestring = ""

In [ ]:
skip_video_for_run_all = True #@param {type: 'boolean'}
fps = 30 #@param {type:"number"}
#@markdown **Manual Settings**
use_manual_settings = False #@param {type:"boolean"}
image_path = "/content/drive/MyDrive/AI/StableDiffusion/2022-09/20220903000939_%05d.png" #@param {type:"string"}
mp4_path = "/content/drive/MyDrive/AI/StableDiffu'/content/drive/MyDrive/AI/StableDiffusion/2022-09/sion/2022-09/20220903000939.mp4" #@param {type:"string"}
render_steps = True  #@param {type: 'boolean'}
path_name_modifier = "x0_pred" #@param ["x0_pred","x"]


if skip_video_for_run_all == True:
    print('Skipping video creation, uncheck skip_video_for_run_all if you want to run it')
else:
    import os
    import subprocess
    from base64 import b64encode

    print(f"{image_path} -> {mp4_path}")

    if use_manual_settings:
        max_frames = "200" #@param {type:"string"}
    else:
        if render_steps: # render steps from a single image
            fname = f"{path_name_modifier}_%05d.png"
            all_step_dirs = [os.path.join(args.outdir, d) for d in os.listdir(args.outdir) if os.path.isdir(os.path.join(args.outdir,d))]
            newest_dir = max(all_step_dirs, key=os.path.getmtime)
            image_path = os.path.join(newest_dir, fname)
            print(f"Reading images from {image_path}")
            mp4_path = os.path.join(newest_dir, f"{args.timestring}_{path_name_modifier}.mp4")
            max_frames = str(args.steps)
        else: # render images for a video
            image_path = os.path.join(args.outdir, f"{args.timestring}_%05d.png")
            mp4_path = os.path.join(args.outdir, f"{args.timestring}.mp4")
            max_frames = str(anim_args.max_frames)

    # make video
    cmd = [
        'ffmpeg',
        '-y',
        '-vcodec', 'png',
        '-r', str(fps),
        '-start_number', str(0),
        '-i', image_path,
        '-frames:v', max_frames,
        '-c:v', 'libx264',
        '-vf',
        f'fps={fps}',
        '-pix_fmt', 'yuv420p',
        '-crf', '17',
        '-preset', 'veryfast',
        '-pattern_type', 'sequence',
        mp4_path
    ]
    process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout, stderr = process.communicate()
    if process.returncode != 0:
        print(stderr)
        raise RuntimeError(stderr)

    mp4 = open(mp4_path,'rb').read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
    display.display( display.HTML(f'<video controls loop><source src="{data_url}" type="video/mp4"></video>') )

# Disconnect when finished

In [ ]:
skip_disconnect_for_run_all = True #@param {type: 'boolean'}

if skip_disconnect_for_run_all == True:
    print('Skipping disconnect, uncheck skip_disconnect_for_run_all if you want to run it')
else:
    from google.colab import runtime
    runtime.unassign()